# Pipeline desctipion

1. Import resources to :
Github/resources
\
    V\
    V\
    V\
    V\
resources/
  *.csv

## Imports

We are using a typical data science stack: `numpy`, `pandas`, `sklearn`, `matplotlib`.

In [20]:
pip install -r requirements.txt

  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [22]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# File system manangement
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

import urllib.request

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import BinaryEncoder, HashingEncoder, LeaveOneOutEncoder, WOEEncoder, PolynomialEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
import numpy as np
import gc  # Garbage collection
from tqdm import tqdm
import os
import json
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import logging
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
import dask.dataframe as dd
from collections import Counter, defaultdict
from dask.diagnostics import ProgressBar
import pyarrow as pa
import pyarrow.csv as pacsv
import pyarrow.parquet as pq
import missingno as msno
import sweetviz as sv
from dataprep.eda import create_report
import argparse
from sklearn.preprocessing import StandardScaler
from termcolor import colored
from collections import defaultdict, Counter
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import category_encoders as ce
from concurrent.futures import ThreadPoolExecutor


## Constantes (Settings)

In [23]:
# Resources
GITHUB_RESOURCES_FOLDER_PATH='https://github.com/lessons-data-ai-engineer/project_4-home_credit_default_risk/tree/main/resources'
LOCAL_RESOURCES_FOLDER_PATH='/content/resources'
# Resources > Files paths
APP_TRAIN_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/application_train.csv'
PREVIOUS_APPLICATION_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/previous_application.csv'
BUREAU_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/bureau.csv'
BUREAU_BALANCE_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/bureau_balance.csv'
POS_CASH_BALANCE_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/POS_CASH_balance.csv'
CREDIT_CARD_BALANCE_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/credit_card_balance.csv'
INSTALLMENTS_PAYMENTS_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/installments_payments.csv'
APP_TEST_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/application_test.csv'
HOMECREDIT_COLUMNS_DESCRIPTION_FILE_PATH = LOCAL_RESOURCES_FOLDER_PATH+'/HomeCredit_columns_description.csv'
# Exports
LOCAL_EXPORT_FOLDER_PATH='/content/exports'
# Export > General Settings
TESTING_MODE=True
TESTING_MODE_MAX_LINES=1000
TESTING_MODE_SUB_FOLDER_NAME='testing_data'
GENERAL_CHUNK_SIZE=10000
# Exports > Columns metadata files
LOCAL_EXPORT_COLUMNS_METADATA_FILES_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/columns_metadata_files'
# Exports > Steps paths > Generate first sweetviz report
LOCAL_EXPORT_FIRST_SWEETVIZ_REPORTS_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/fist_sweetviz_reports'
# Exports > Steps paths > missing_values_replaced
LOCAL_EXPORT_MISSING_VALUES_REPLACED_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/missing_values_replaced'
# Exports > Steps paths > columns_types_checked
LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/columns_types_checked'
# Exports > Steps paths > Replace to nan
LOCAL_EXPORT_REPLACE_EMPTY_TO_NAN_PATH=LOCAL_EXPORT_FOLDER_PATH+'/replace_to_nan'
# Exports > Steps paths > Imputation
LOCAL_EXPORT_IMPUTATION_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/imputed'
# Exports > Steps paths > Imputation checks
LOCAL_EXPORT_IMPUTATION_CHECKS_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/imputed_checks'
# Exports > Steps paths > Imputation evaluation
LOCAL_EXPORT_IMPUTATION_RESULTS_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/imputed_results'
# Exports > Steps paths > Without outliers
LOCAL_EXPORT_IMPUTED_WITHOUT_OUTLIERS_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/imputed_without_outliers'
# Exports > Steps paths > Without duplicates
LOCAL_EXPORT_IMPUTED_WITHOUT_DUPLICATES_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/imputed_without_duplicates'
# Exports > Steps paths > Feature Enginering
LOCAL_EXPORT_FEATURE_ENGINEERED_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/feature_engineered'
# Export > Steps paths > Merge dataframes
LOCAL_EXPORT_MERGED_DATAFRAMES_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/merged_dataframes'
# Export > Steps paths > Encoded & Aligned
LOCAL_EXPORT_ENCODED_AND_ALIGNED_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/encoded_and_aligned'
# Export > Steps paths > Manual verification after encoding
LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH=LOCAL_EXPORT_FOLDER_PATH+'/manual_check_patch'


## Resources import

In [ ]:
# Créer les dossiers locaux si nécessaire
os.makedirs(LOCAL_RESOURCES_FOLDER_PATH, exist_ok=True)
os.makedirs(LOCAL_EXPORT_FOLDER_PATH, exist_ok=True)

# Télécharger la liste des fichiers depuis GitHub
!wget -O {LOCAL_RESOURCES_FOLDER_PATH}/index.html {GITHUB_RESOURCES_FOLDER_PATH}

# Extraire les liens des fichiers CSV
!grep -oP 'href="\K[^"]+\.csv' {LOCAL_RESOURCES_FOLDER_PATH}/index.html | awk '{print "https://github.com"$1}' > {LOCAL_RESOURCES_FOLDER_PATH}/csv_links.txt

# Extraire les noms de fichiers CSV de la liste des liens
csv_files = []
with open(f'{LOCAL_RESOURCES_FOLDER_PATH}/csv_links.txt', 'r') as f:
    for line in f:
        file_name = line.strip().split('/')[-1]
        csv_files.append(file_name)

# Liste des fichiers existants dans le dossier local
local_files = os.listdir(LOCAL_RESOURCES_FOLDER_PATH)

# Supprimer les fichiers locaux qui ne sont pas dans la liste des fichiers CSV sur GitHub
for local_file in local_files:
    if local_file not in csv_files and local_file.endswith('.csv'):
        os.remove(os.path.join(LOCAL_RESOURCES_FOLDER_PATH, local_file))
        print(f"Deleted: {local_file}")

# Télécharger chaque fichier CSV sans duplication et mettre à jour les fichiers existants
with open(f'{LOCAL_RESOURCES_FOLDER_PATH}/csv_links.txt', 'r') as f:
    for line in f:
        # Modifier l'URL pour obtenir le lien de téléchargement direct
        download_url = line.strip().replace('/blob/', '/raw/')
        # Utiliser -N pour éviter la duplication des fichiers et mettre à jour les fichiers existants
        !wget -P {LOCAL_RESOURCES_FOLDER_PATH} -N {download_url}

# Vérifier les fichiers téléchargés
!ls {LOCAL_RESOURCES_FOLDER_PATH}

## Resources check

First, we can list all the available data files. There are a total of 9 files: 1 main file for training (with target) 1 main file for testing (without the target), 1 example submission file, and 6 other files containing additional information about each loan.

In [24]:
# List files available
print(os.listdir(LOCAL_RESOURCES_FOLDER_PATH))

['sample_submission.csv', 'application_test.csv', 'POS_CASH_balance.csv', 'installments_payments.csv', 'bureau_balance.csv', 'previous_application.csv', 'bureau.csv', 'credit_card_balance.csv', 'application_train.csv', 'HomeCredit_columns_description.csv']


## Dataframes creation

In [ ]:
app_train = pd.read_csv(APP_TRAIN_FILE_PATH)
previous_application = pd.read_csv(PREVIOUS_APPLICATION_FILE_PATH)
bureau = pd.read_csv(BUREAU_FILE_PATH)
bureau_balance = pd.read_csv(BUREAU_BALANCE_FILE_PATH)
POS_CASH_balance = pd.read_csv(POS_CASH_BALANCE_FILE_PATH)
credit_card_balance = pd.read_csv(CREDIT_CARD_BALANCE_FILE_PATH)
installments_payments = pd.read_csv(INSTALLMENTS_PAYMENTS_FILE_PATH)
app_test = pd.read_csv(APP_TEST_FILE_PATH)
HomeCredit_columns_description = pd.read_csv(HOMECREDIT_COLUMNS_DESCRIPTION_FILE_PATH,encoding='ISO-8859-1')

In [42]:
app_test = pd.read_csv(APP_TEST_FILE_PATH)

In [38]:
HomeCredit_columns_description = pd.read_csv(HOMECREDIT_COLUMNS_DESCRIPTION_FILE_PATH,encoding='ISO-8859-1')

The training data has 307511 observations (each one a separate loan) and 122 features (variables) including the `TARGET` (the label we want to predict).

## Dataframes preview

In [ ]:
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
print('previous_application data shape: ', previous_application.shape)
previous_application.head()

In [ ]:
print('bureau data shape: ', bureau.shape)
bureau.head()

In [ ]:
print('bureau_balance data shape: ', bureau_balance.shape)
bureau_balance.head()

In [ ]:
print('POS_CASH_balance data shape: ', POS_CASH_balance.shape)
POS_CASH_balance.head()

In [ ]:
print('credit_card_balance data shape: ', credit_card_balance.shape)
credit_card_balance.head()

In [ ]:
print('installments_payments data shape: ', installments_payments.shape)
installments_payments.head()

In [43]:
print('Testing data shape: ', app_test.shape)
app_test.head()

Testing data shape:  (2, 1)


,version https://git-lfs.github.com/spec/v1
0,oid sha256:a36161331d839150a67b6216d4de066f543...
1,size 26567651


In [40]:
print('Testing data shape: ', HomeCredit_columns_description.shape)
HomeCredit_columns_description.head()

Testing data shape:  (2, 1)


,version https://git-lfs.github.com/spec/v1
0,oid sha256:eef7665398228a80f7367c9258220c5fbe1...
1,size 37383


The test set is considerably smaller and lacks a `TARGET` column.

# Exploratory Data Analysis

Exploratory Data Analysis (EDA) is an open-ended process where we calculate statistics and make figures to find trends, anomalies, patterns, or relationships within the data. The goal of EDA is to learn what our data can tell us. It generally starts out with a high level overview, then narrows in to specific areas as we find intriguing areas of the data. The findings may be interesting in their own right, or they can be used to inform our modeling choices, such as by helping us decide which features to use.

## Examine the Distribution of the Target Column

The target is what we are asked to predict: either a 0 for the loan was repaid on time, or a 1 indicating the client had payment difficulties. We can first examine the number of loans falling into each category.

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist();

From this information, we see this is an [_imbalanced class problem_](http://www.chioka.in/class-imbalance-problem/). There are far more loans that were repaid on time than loans that were not repaid. Once we get into more sophisticated machine learning models, we can [weight the classes](http://xgboost.readthedocs.io/en/latest/parameter.html) by their representation in the data to reflect this imbalance.

## Examine Missing Values

Next we can look at the number and percentage of missing values in each column.

In [25]:
def missing_values_table(df, df_name):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Add a new column with the name of the dataframe
    mis_val_table_ren_columns['DataFrame Name'] = df_name

    # Correctly rename the index
    mis_val_table_ren_columns = mis_val_table_ren_columns.rename_axis('Row')

    # Sort the table by percentage of missing descending, including those with 0% missing
    mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values(
        '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(mis_val_table_ren_columns[mis_val_table_ren_columns['Missing Values'] > 0].shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values_app_train = missing_values_table(app_train,'application_{train|test}.csv')
missing_values_bureau = missing_values_table(bureau, 'bureau.csv')
missing_values_bureau_balance = missing_values_table(bureau_balance, 'bureau_balance.csv')
missing_values_POS_CASH_balance = missing_values_table(POS_CASH_balance, 'POS_CASH_balance.csv')
missing_values_credit_card_balance = missing_values_table(credit_card_balance, 'credit_card_balance.csv')
missing_values_previous_application = missing_values_table(previous_application, 'previous_application.csv')
missing_values_installments_payments = missing_values_table(installments_payments, 'installments_payments.csv')
missing_values_app_train.head(20)

In [ ]:
concat_columns = pd.concat([
    missing_values_app_train,
    missing_values_bureau,
    missing_values_bureau_balance,
    missing_values_POS_CASH_balance,
    missing_values_credit_card_balance,
    missing_values_previous_application,
    missing_values_installments_payments], axis=0)

In [ ]:
# concat_columns_grouped = concat_columns.groupby('Row').sum()
concat_columns.head(200)

In [ ]:
hccd = HomeCredit_columns_description[["Row", "Description"]]
# result = pd.merge(hccd, missing_values, on="Row", how="inner")

result_all_columns = pd.merge(hccd, concat_columns, on="Row", how="inner")
result_all_columns.head(71)
result_all_columns.to_csv('hccd_and_missing_values_all_dfs.csv', index=True)

In [ ]:
LOCAL_EXPORT_COLUMNS_METADATA_FILES_FOLDER_PATH

When it comes time to build our machine learning models, we will have to fill in these missing values (known as imputation). In later work, we will use models such as XGBoost that can [handle missing values with no need for imputation](https://stats.stackexchange.com/questions/235489/xgboost-can-handle-missing-data-in-the-forecasting-phase). Another option would be to drop columns with a high percentage of missing values, although it is impossible to know ahead of time if these columns will be helpful to our model. Therefore, we will keep all of the columns for now.

## Columns description & missing values studings

### Method

In [35]:
def analyze_missing_values_and_columns_description(input_folder, output_folder, columns_description_file, main_file, files_with_target=None, target_column=None, analyze_columns=None, extract_columns=None, testing=False, max_rows=1000, exclude_files=None, testing_output_folder="testing_data", chunksize=50000):
    """
    Analyze missing values for all CSV files in the input folder, and save reports in the output folder.
    """

    def missing_values_table(df, df_name):
        """
        Create a table with the number and percentage of missing values for each column of the DataFrame.
        """
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})
        mis_val_table_ren_columns['DataFrame Name'] = df_name
        mis_val_table_ren_columns.index.name = 'Row'
        mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values('% of Total Values', ascending=False).round(1)
        return mis_val_table_ren_columns

    def column_metadata(df, df_name, target_column=None):
        """
        Create a table with metadata for each column of the DataFrame, including correlation with the target column.
        """
        metadata = []
        for col in df.columns:
            corr = None
            if target_column and target_column in df.columns and col != target_column:
                if df[col].dtype in [float, int]:
                    if df[col].var() != 0 and df[col].notnull().sum() > 1:
                        try:
                            corr = df[col].corr(df[target_column])
                        except FloatingPointError:
                            corr = None
            col_type = df[col].dtype
            unique_vals = df[col].nunique()
            metadata.append({
                'Row': col,
                'Corr_Target_Before_Cleaning': corr,
                'Data_Type': col_type,
                'Unique_Values': unique_vals,
                'DataFrame Name': df_name
            })
        metadata_df = pd.DataFrame(metadata)
        metadata_df.set_index('Row', inplace=True)
        return metadata_df

    # Load the column descriptions
    hccd = pd.read_csv(os.path.join(input_folder, columns_description_file), encoding="ISO-8859-1")

    # Check that 'Row' and 'DataFrame Name' are in hccd
    if 'Row' not in hccd.columns or 'DataFrame Name' not in hccd.columns:
        raise KeyError("The columns 'Row' and 'DataFrame Name' must be present in the columns description file.")

    # Load the main file containing the target column
    main_df = pd.read_csv(os.path.join(input_folder, main_file))

    # Check if the output folder exists, otherwise create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # If in testing mode, create the subfolder for testing data
    if testing:
        testing_output_folder = os.path.join(output_folder, testing_output_folder)
        if not os.path.exists(testing_output_folder):
            os.makedirs(testing_output_folder)

    # List all CSV files in the input folder
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    # Exclude specified files
    if exclude_files:
        csv_files = [f for f in csv_files if f not in exclude_files]

    # Initialize lists to store missing values and metadata results
    missing_values_list = []
    metadata_list = []

    # Iterate through the CSV files with a progress bar
    for filename in tqdm(csv_files, desc="Analyzing missing values and column metadata", unit="file"):
        if filename == main_file:
            continue  # The main file is already loaded

        file_path = os.path.join(input_folder, filename)
        print(f"\nAnalyzing {file_path}...")

        # Load data in chunks
        chunk_iter = pd.read_csv(file_path, chunksize=chunksize)

        for chunk in chunk_iter:
            if testing:
                chunk = chunk.head(max_rows)

            if extract_columns:
                chunk = chunk[extract_columns]

            if 'SK_ID_CURR' in chunk.columns:
                chunk = chunk.merge(main_df[['SK_ID_CURR', target_column]], on='SK_ID_CURR', how='left')
            elif 'SK_ID_BUREAU' in chunk.columns:
                chunk = chunk.merge(main_df[['SK_ID_CURR', target_column]], left_on='SK_ID_BUREAU', right_on='SK_ID_CURR', how='left')
            elif 'SK_ID_PREV' in chunk.columns:
                chunk = chunk.merge(main_df[['SK_ID_CURR', target_column]], left_on='SK_ID_PREV', right_on='SK_ID_CURR', how='left')

            missing_values = missing_values_table(chunk, filename)
            missing_values_list.append(missing_values)

            if files_with_target and filename in files_with_target:
                metadata = column_metadata(chunk, filename, target_column)
            else:
                metadata = column_metadata(chunk, filename)

            metadata_list.append(metadata)

    concat_missing_values = pd.concat(missing_values_list, axis=0)
    concat_metadata = pd.concat(metadata_list, axis=0)

    concat_missing_values.reset_index(inplace=True)
    concat_metadata.reset_index(inplace=True)

    print("Missing Values DataFrame Columns:", concat_missing_values.columns)
    print("Metadata DataFrame Columns:", concat_metadata.columns)

    # Check if 'Row' and 'DataFrame Name' columns exist before merging
    if 'Row' not in concat_missing_values.columns or 'Row' not in concat_metadata.columns:
        raise KeyError("The 'Row' column is missing in one of the dataframes.")
    if 'DataFrame Name' not in concat_missing_values.columns or 'DataFrame Name' not in concat_metadata.columns:
        raise KeyError("The 'DataFrame Name' column is missing in one of the dataframes.")

    # Merge with column descriptions using 'Row' and 'DataFrame Name'
    result_all_columns = pd.merge(hccd, concat_missing_values, on=["Row", "DataFrame Name"], how="inner")
    result_all_columns = pd.merge(result_all_columns, concat_metadata, on=["Row", "DataFrame Name"], how="inner")

    if testing:
        missing_values_output_file = os.path.join(testing_output_folder, 'hccd_and_missing_values_all_dfs.csv')
    else:
        missing_values_output_file = os.path.join(output_folder, 'hccd_and_missing_values_all_dfs.csv')

    result_all_columns.to_csv(missing_values_output_file, index=True)
    print(f"Missing values and column metadata analysis saved to {missing_values_output_file}")

### Call

In [36]:
analyze_missing_values_and_columns_description(
        input_folder=LOCAL_RESOURCES_FOLDER_PATH,
        output_folder=LOCAL_EXPORT_COLUMNS_METADATA_FILES_FOLDER_PATH,
        main_file='application_train.csv',
        chunksize=GENERAL_CHUNK_SIZE,
        files_with_target=["application_train.csv", "previous_application.csv"],
        target_column='TARGET',
        columns_description_file='HomeCredit_columns_description.csv',
        analyze_columns=None,
        extract_columns=None,
        testing=TESTING_MODE,
        max_rows=TESTING_MODE_MAX_LINES,
        exclude_files=['application_test.csv','HomeCredit_columns_description.csv','sample_submission.csv','credit_card_balance.csv'],
        testing_output_folder=TESTING_MODE_SUB_FOLDER_NAME
  )

KeyError: "The columns 'Row' and 'DataFrame Name' must be present in the columns description file."

In [ ]:
hccd_and_missing_values_all_dfs = pd.read_csv(LOCAL_EXPORT_COLUMNS_METADATA_FILES_FOLDER_PATH+'/'+TESTING_MODE_SUB_FOLDER_NAME+'/' +'hccd_and_missing_values_all_dfs.csv')

In [ ]:
hccd_and_missing_values_all_dfs.head(70)

## First analyse reports for all dfs

### Method

In [ ]:
def generate_sweetviz_reports(input_folder, output_folder, analyze_columns=None, extract_columns=None, testing=False, max_lines=1000, exclude_files=None, testing_output_subfolder="testing_data"):
    """
    Génère des rapports Sweetviz pour tous les fichiers CSV dans le dossier d'entrée
    et enregistre les rapports dans le dossier de sortie.

    Parameters:
    input_folder (str): Le chemin du dossier contenant les fichiers CSV.
    output_folder (str): Le chemin du dossier où les rapports HTML seront enregistrés.
    analyze_columns (list): Liste des colonnes à analyser.
    extract_columns (list): Liste des colonnes à extraire.
    testing (bool): Mode testing pour analyser seulement les max_lines premières lignes.
    max_lines (int): Nombre maximum de lignes à analyser en mode testing.
    exclude_files (list): Liste des fichiers à exclure de l'analyse.
    testing_output_subfolder (str): Nom du sous-dossier pour les données de test en mode testing.
    """
    # Déterminer le dossier de sortie final
    if testing:
        output_folder = os.path.join(output_folder, testing_output_subfolder)

    # Vérifier si le dossier de sortie existe, sinon le créer
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lister tous les fichiers CSV dans le dossier d'entrée
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    # Exclure les fichiers spécifiés
    if exclude_files:
        csv_files = [f for f in csv_files if f not in exclude_files]

    # Parcourir les fichiers CSV avec une barre de progression
    for filename in tqdm(csv_files, desc="Generating Sweetviz reports", unit="file"):
        file_path = os.path.join(input_folder, filename)
        print(f"\nAnalyzing {file_path}...")

        # Charger les données
        df = pd.read_csv(file_path)

        # Mode testing : limiter le nombre de lignes
        if testing:
            df = df.head(max_lines)

        # Appliquer les colonnes à extraire si spécifiées
        if extract_columns:
            df = df[extract_columns]

        # Générer le rapport
        if analyze_columns:
            report = sv.analyze(df[analyze_columns])
        else:
            report = sv.analyze(df)

        # Chemin du fichier de sortie
        output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_sweetviz_report.html")

        # Enregistrer le rapport en HTML
        report.show_html(output_file_path, open_browser=False)
        print(f"Report saved to {output_file_path}")

### Call

In [ ]:
 generate_sweetviz_reports(
     input_folder=LOCAL_RESOURCES_FOLDER_PATH,
     output_folder=LOCAL_EXPORT_FIRST_SWEETVIZ_REPORTS_FOLDER_PATH,
     analyze_columns=None,
     extract_columns=None,
     testing=TESTING_MODE,
     max_lines=TESTING_MODE_MAX_LINES,
     exclude_files=['HomeCredit_columns_description.csv','sample_submission.csv'],
     testing_output_subfolder=TESTING_MODE_SUB_FOLDER_NAME)

## Replace empty values by NaN

### Method

In [ ]:
def load_and_preprocess_dataframes(input_folder, export_folder, chunk_size=100000, ignore_files=None, max_lines=None, testing=False, testing_subfolder="testing_data"):
    """
    Charge plusieurs DataFrames à partir de fichiers CSV dans un dossier, remplace les valeurs vides par NaN,
    et exporte les DataFrames corrigés dans un autre dossier.

    :param input_folder: Dossier contenant les fichiers CSV à charger
    :param export_folder: Dossier où les DataFrames corrigés seront exportés
    :param chunk_size: Taille des chunks pour le traitement des DataFrames
    :param ignore_files: Liste de fichiers à ignorer lors du traitement
    :param max_lines: Nombre maximum de lignes à traiter par fichier (uniquement pris en compte si testing est True)
    :param testing: Booléen indiquant si le mode test est activé
    :param testing_subfolder: Nom du sous-dossier pour les données de test en mode testing
    :return: Dictionnaire des DataFrames corrigés
    """
    if ignore_files is None:
        ignore_files = []

    if testing:
        export_folder = os.path.join(export_folder, testing_subfolder)

    if not os.path.exists(export_folder):
        os.makedirs(export_folder)

    # Lister tous les fichiers CSV dans le dossier d'entrée, en excluant les fichiers ignorés
    csv_files = {os.path.splitext(file)[0]: os.path.join(input_folder, file)
                 for file in os.listdir(input_folder) if file.endswith('.csv') and file not in ignore_files}

    for key, path in csv_files.items():
        corrected_file_path = os.path.join(export_folder, f"{key}.csv")
        total_lines_processed = 0

        try:
            with pd.read_csv(path, chunksize=chunk_size) as reader:
                for i, chunk in enumerate(reader):
                    if testing and max_lines is not None and total_lines_processed >= max_lines:
                        break

                    if testing and max_lines is not None:
                        lines_to_process = chunk.head(max_lines - total_lines_processed)
                    else:
                        lines_to_process = chunk

                    total_lines_processed += len(lines_to_process)

                    lines_to_process.replace(['', ' ', 'NA', 'N/A', None, 'NaN'], np.nan, inplace=True)
                    mode = 'w' if i == 0 else 'a'
                    header = i == 0
                    lines_to_process.to_csv(corrected_file_path, mode=mode, header=header, index=False)
                    print(f"{key}: chunk {i + 1} processed and saved to {corrected_file_path}")

            print(f"Completed processing for {key}.")
        except Exception as e:
            print(f"Failed to process {path} due to {e}")

    print(f"All files processed and saved to {export_folder}")

### Call

In [ ]:
load_and_preprocess_dataframes(
    input_folder=LOCAL_RESOURCES_FOLDER_PATH,
    export_folder=LOCAL_EXPORT_MISSING_VALUES_REPLACED_FOLDER_PATH,
    chunk_size=GENERAL_CHUNK_SIZE,
    ignore_files=["HomeCredit_columns_description.csv",'sample_submission.csv'],
    max_lines=TESTING_MODE_MAX_LINES,
    testing=TESTING_MODE,
    testing_subfolder=TESTING_MODE_SUB_FOLDER_NAME
)

## Columns types verification

### Method

In [ ]:
# Configure logging
logging.basicConfig(filename='data_processing.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s')

def analyze_chunk_types(chunk):
    type_summary = defaultdict(Counter)
    for column in chunk.columns:
        type_counts = chunk[column].apply(detect_type).value_counts(normalize=True)
        for dtype, percentage in type_counts.items():
            type_summary[column][dtype] += percentage * len(chunk) / len(chunk.index)
    return type_summary

def detect_type(value):
    if pd.isnull(value):
        return 'null'
    try:
        int(value)
        return 'int'
    except ValueError:
        pass
    try:
        float(value)
        return 'float'
    except ValueError:
        pass
    try:
        pd.to_datetime(value)
        return 'datetime'
    except (ValueError, TypeError):
        pass
    if str(value).lower() in ['true', 'false']:
        return 'bool'
    return 'str'

def merge_type_summaries(main_summary, chunk_summary):
    for column, types in chunk_summary.items():
        for dtype, percentage in types.items():
            main_summary[column][dtype] += percentage

def normalize_type_summaries(type_summary, total_chunks):
    if total_chunks == 0:
        return type_summary
    for column, types in type_summary.items():
        for dtype in types:
            type_summary[column][dtype] /= total_chunks
    return type_summary

def display_type_summary(file_name, type_summary):
    logging.info(f"\nFile: {file_name}")
    for column, types in type_summary.items():
        logging.info(f"Column: {column}")
        for dtype, percentage in types.items():
            logging.info(f"  {dtype}: {percentage:.2%}")

def get_most_frequent_type(column_types):
    if not column_types:
        return None
    return max(column_types, key=column_types.get)

def convert_column_types(data, type_summary, identifier_columns):
    for column, types in type_summary.items():
        if column in identifier_columns:
            data[column] = data[column].astype(str)
            continue

        most_frequent_type = get_most_frequent_type(types)
        if most_frequent_type:
            try:
                if most_frequent_type == 'int':
                    data[column] = pd.to_numeric(data[column], errors='coerce')
                    data[column] = data[column].fillna(0).astype('Int64')
                elif most_frequent_type == 'float':
                    data[column] = pd.to_numeric(data[column], errors='coerce')
                elif most_frequent_type == 'bool':
                    data[column] = data[column].astype(bool)
                elif most_frequent_type == 'datetime':
                    data[column] = pd.to_datetime(data[column], errors='coerce')
                elif most_frequent_type == 'str':
                    data[column] = data[column].astype(str)
                elif most_frequent_type == 'categorical':
                    if data[column].nunique() < len(data[column]) * 0.05:
                        data[column] = data[column].astype('category')
            except Exception as e:
                logging.error(f"Error converting column {column} to {most_frequent_type}: {e}")
    return data

def generate_report(file_name, type_summary_before, type_summary_after):
    report = f"\nFile: {file_name}\n"
    report += "Column Type Summary Before Conversion:\n"
    for column, types in type_summary_before.items():
        report += f"  Column: {column}\n"
        for dtype, percentage in types.items():
            report += f"    {dtype}: {percentage:.2%}\n"
    report += "Column Type Summary After Conversion:\n"
    for column, types in type_summary_after.items():
        report += f"  Column: {column}\n"
        for dtype in types:
            report += f"    {dtype}\n"
    return report

def save_type_summary_report(file_name, type_summary):
    report = f"Type Summary for {file_name}:\n"
    for column, types in type_summary.items():
        report += f"  {column}:\n"
        for dtype, percentage in types.items():
            report += f"    {dtype}: {percentage:.2%}\n"
    with open(file_name, 'w') as f:
        f.write(report)

def load_and_preprocess_dataframes(input_folder, export_folder, chunk_size=10000, ignore_files=None, max_rows=None, testing=False, testing_sub_folder_name="testing_data", identifier_columns=None):
    if ignore_files is None:
        ignore_files = []

    if identifier_columns is None:
        identifier_columns = []

    if testing:
        input_folder = os.path.join(input_folder, testing_sub_folder_name)
        export_folder = os.path.join(export_folder, testing_sub_folder_name)

    if not os.path.exists(export_folder):
        os.makedirs(export_folder)

    csv_files = {os.path.splitext(file)[0]: os.path.join(input_folder, file)
                 for file in os.listdir(input_folder) if file.endswith('.csv') and file not in ignore_files}

    for key, path in csv_files.items():
        corrected_file_path = os.path.join(export_folder, f"{key}.csv")
        total_lines_processed = 0
        type_summary = defaultdict(Counter)
        chunk_count = 0

        try:
            with pd.read_csv(path, chunksize=chunk_size) as reader:
                for i, chunk in enumerate(reader):
                    if testing and max_rows is not None and total_lines_processed >= max_rows:
                        break

                    if testing and max_rows is not None:
                        lines_to_process = chunk.head(max_rows - total_lines_processed)
                    else:
                        lines_to_process = chunk

                    total_lines_processed += len(lines_to_process)
                    chunk_count += 1

                    chunk_types = analyze_chunk_types(lines_to_process)
                    merge_type_summaries(type_summary, chunk_types)

                    lines_to_process.replace(['', ' ', 'NA', 'N/A', None, 'NaN'], np.nan, inplace=True)
                    mode = 'w' if i == 0 else 'a'
                    header = i == 0
                    lines_to_process.to_csv(corrected_file_path, mode=mode, header=header, index=False)
                    logging.info(f"{key}: chunk {i + 1} processed and saved to {corrected_file_path}")

            # Normalize type summary
            normalized_type_summary = normalize_type_summaries(type_summary, chunk_count)

            # Load the entire file to convert types
            data = pd.read_csv(corrected_file_path)
            data = convert_column_types(data, normalized_type_summary, identifier_columns)
            data.to_csv(corrected_file_path, index=False)

            # Generate and log the report
            final_type_summary = analyze_chunk_types(data)
            report = generate_report(key, normalized_type_summary, final_type_summary)
            logging.info(report)

            # Save the type summary report to a file
            report_file_path = os.path.join(export_folder, f"{key}_type_summary.txt")
            save_type_summary_report(report_file_path, final_type_summary)

            logging.info(f"Completed processing and profiling for {key}.")
        except Exception as e:
            logging.error(f"Failed to process {path} due to {e}")

    logging.info(f"All files processed and saved to {export_folder}")


### Call

In [ ]:
load_and_preprocess_dataframes(
    input_folder=LOCAL_EXPORT_MISSING_VALUES_REPLACED_FOLDER_PATH,
    export_folder=LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH,
    chunk_size=GENERAL_CHUNK_SIZE,
    ignore_files=None,
    max_rows=TESTING_MODE_MAX_LINES,
    testing=TESTING_MODE,
    testing_sub_folder_name=TESTING_MODE_SUB_FOLDER_NAME,
    identifier_columns=["SK_ID_PREV", "SK_ID_CURR","SK_ID_BUREAU"])

In [ ]:
test = pd.read_csv(LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH+"/testing_data/application_train.csv")

In [ ]:
test.head()

## Imputation

### Imputation methods

In [ ]:
# Logger class as provided by you
class Logger:
    def __init__(self, level=4, export_logs=False, log_folder_path='logs'):
        self.log_level = level
        self.export_logs = export_logs
        self.log_folder_path = log_folder_path
        self.log_messages = []

    def log(self, message, level=4, color=None):
        if level <= self.log_level:
            if color:
                print(colored(message, color))
            else:
                print(message)
            self.log_messages.append(message)

    def export(self, log_file_name='logs.txt'):
        if self.export_logs:
            log_file_path = os.path.join(self.log_folder_path, log_file_name)
            os.makedirs(self.log_folder_path, exist_ok=True)
            with open(log_file_path, 'w') as log_file:
                for message in self.log_messages:
                    log_file.write(message + '\n')
            print(colored(f'Logs exported to {log_file_path}', 'green'))

# Save original data types of columns
def save_column_types(df):
    return df.dtypes.to_dict()

# Restore data types of columns
def restore_column_types(df, column_types):
    for column, dtype in column_types.items():
        df[column] = df[column].astype(dtype)
    return df

def prepare_data_for_imputation(df, logger, exclude_columns=None, missing_threshold=0.8, extreme_value_threshold=1e15, small_value_threshold=1e-15, round_decimals=None):
    if exclude_columns is None:
        exclude_columns = []
    logger.log("Starting data preparation", 2)

    # Initial data stats
    logger.log(f"Initial dataframe shape: {df.shape}", 3)
    logger.log(f"Initial columns with missing values above {missing_threshold * 100}%:", 3)
    for col in df.columns:
        if col in exclude_columns:
            continue
        missing_ratio = df[col].isnull().mean()
        if missing_ratio > missing_threshold:
            logger.log(f"Column {col} has {missing_ratio * 100:.2f}% missing values", 4)

    # Drop columns with missing values above the threshold
    missing_ratio = df.isnull().mean()
    cols_to_drop = missing_ratio[missing_ratio > missing_threshold].index
    df = df.drop(columns=[col for col in cols_to_drop if col not in exclude_columns])
    logger.log(f"Dropped columns: {list(set(cols_to_drop) - set(exclude_columns))}", 3)
    logger.log(f"Dataframe shape after dropping columns: {df.shape}", 3)

    # Cap extreme values and replace small values to avoid underflow
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col in exclude_columns:
            continue
        col_min = df[col].min()
        col_max = df[col].max()
        logger.log(f"Column {col}: min before capping = {col_min}, max before capping = {col_max}", 4)

        df[col] = np.clip(df[col], -extreme_value_threshold, extreme_value_threshold)
        df[col] = df[col].apply(lambda x: small_value_threshold if 0 < abs(x) < small_value_threshold else x)

        col_min_after = df[col].min()
        col_max_after = df[col].max()
        logger.log(f"Column {col}: min after capping = {col_min_after}, max after capping = {col_max_after}", 4)

    # Replace infinite values with NaN
    df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)

    # Ensure no duplicated indices
    df = df[~df.index.duplicated(keep='first')]

    # Convert all numeric columns to float64 to ensure uniformity
    df[numeric_cols] = df[numeric_cols].astype(np.float64)

    # Round numerical values if specified
    if round_decimals is not None:
        df[numeric_cols] = df[numeric_cols].round(round_decimals)
        logger.log(f"Rounded numerical values to {round_decimals} decimal places", 3)

    # Log final data stats
    for col in numeric_cols:
        logger.log(f"Column {col}: final min = {df[col].min()}, final max = {df[col].max()}, final NaNs = {df[col].isnull().sum()}", 3)

    logger.log("Data preparation completed", 2, "green")
    return df

def impute_and_save_dataframes_from_folder(input_folder, imputations, exclude_columns=None, chunk_size=10000, output_folder='LOCAL_EXPORT_FOLDER_PATH/',
                                           clean_memory=False, testing=False, max_rows=None, testing_data_sub_folder='testing_data',
                                           round_decimals=None, log_level=4, export_logs=False, log_folder_path='logs'):
    logger = Logger(level=log_level, export_logs=export_logs, log_folder_path=log_folder_path)

    if testing:
        input_folder = os.path.join(input_folder, testing_data_sub_folder)
        output_folder = os.path.join(output_folder, testing_data_sub_folder)
        logger.log_folder_path = os.path.join(logger.log_folder_path, testing_data_sub_folder)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    csv_files = {os.path.splitext(file)[0]: os.path.join(input_folder, file)
                 for file in os.listdir(input_folder) if file.endswith('.csv')}

    total_tasks = len(csv_files) * len(imputations)
    task_count = 0

    results = {}

    for df_name, csv_file in csv_files.items():
        try:
            logger.log(f"Processing file {csv_file}", 2)
            original_df = pd.read_csv(csv_file)
            logger.log(f"Original dataframe shape: {original_df.shape}", 3)
            if testing and max_rows is not None:
                original_df = original_df.iloc[:max_rows]
                logger.log(f"Testing mode: reduced dataframe shape: {original_df.shape}", 3)

            # Save original column types
            original_column_types = save_column_types(original_df)
            logger.log(f"Original column types: {original_column_types}", 3)

            original_df = prepare_data_for_imputation(original_df, logger, exclude_columns, round_decimals=round_decimals)

            results[df_name] = {
                "original": csv_file,
                "imputed": {}
            }

            for imp_name, imputer in imputations:
                try:
                    logger.log(f"Starting imputation for {df_name} using {imp_name}", 2)
                    df = original_df.copy()
                    imputed_chunks = []
                    for chunk_start in tqdm(range(0, df.shape[0], chunk_size), desc=f'Imputing {df_name} with {imp_name}', leave=False):
                        chunk_end = min(chunk_start + chunk_size, df.shape[0])
                        df_chunk = df.iloc[chunk_start:chunk_end].copy()

                        logger.log(f"Processing chunk: rows {chunk_start} to {chunk_end}", 3)
                        for col in df_chunk.select_dtypes(include=[np.number]).columns:
                            logger.log(f"Chunk {chunk_start}-{chunk_end} - Column {col}: min = {df_chunk[col].min()}, max = {df_chunk[col].max()}, NaNs = {df_chunk[col].isnull().sum()}", 4)

                        numeric_cols = df_chunk.select_dtypes(include=[np.number]).columns
                        non_numeric_cols = df_chunk.select_dtypes(exclude=[np.number]).columns

                        if numeric_cols.size > 0:
                            numeric_imputer = imputer if isinstance(imputer, (SimpleImputer, KNNImputer, IterativeImputer)) else None
                            if numeric_imputer:
                                df_chunk[numeric_cols] = numeric_imputer.fit_transform(df_chunk[numeric_cols])
                            else:
                                df_chunk[numeric_cols] = df_chunk[numeric_cols].apply(imputer)

                        if non_numeric_cols.size > 0:
                            non_numeric_imputer = SimpleImputer(strategy='most_frequent')
                            df_chunk[non_numeric_cols] = non_numeric_imputer.fit_transform(df_chunk[non_numeric_cols])

                        # Verify no NaN in the chunk
                        if df_chunk.isnull().any().any():
                            nan_columns = df_chunk.columns[df_chunk.isnull().any()].tolist()
                            logger.log(f"NaN values found in chunk: rows {chunk_start} to {chunk_end}, columns: {nan_columns}", 1, 'red')
                            logger.log(f"Attempting to fill remaining NaNs with default value", 2, 'yellow')

                            # Fill NaN with 0 or median
                            for col in nan_columns:
                                if df_chunk[col].dtype == np.number:
                                    df_chunk[col].fillna(df_chunk[col].median(), inplace=True)
                                else:
                                    df_chunk[col].fillna(0, inplace=True)

                            # Verify again
                            if df_chunk.isnull().any().any():
                                nan_columns = df_chunk.columns[df_chunk.isnull().any()].tolist()
                                logger.log(f"Still NaN values found in chunk after default filling: rows {chunk_start} to {chunk_end}, columns: {nan_columns}", 1, 'red')
                                raise ValueError(f"NaN values found in the imputed chunk for {df_name} using method {imp_name}")

                        imputed_chunks.append(df_chunk)

                        del df_chunk
                        gc.collect()

                    imputed_df = pd.concat(imputed_chunks, ignore_index=True)

                    # Restore original column types
                    imputed_df = restore_column_types(imputed_df, original_column_types)

                    # Second pass to handle any remaining NaNs
                    remaining_na_cols = imputed_df.columns[imputed_df.isnull().any()].tolist()
                    if remaining_na_cols:
                        logger.log(f"Columns with remaining NaNs after first imputation: {remaining_na_cols}", 3)
                        for col in remaining_na_cols:
                            if imputed_df[col].dtype == np.number:
                                imputed_df[col].fillna(imputed_df[col].median(), inplace=True)
                            else:
                                imputed_df[col].fillna(0, inplace=True)

                    # Blocking condition: raise error if NaNs still exist
                    if imputed_df.isnull().any().any():
                        nan_columns = imputed_df.columns[imputed_df.isnull().any()].tolist()
                        logger.log(f"Blocking error: Columns with remaining NaNs: {nan_columns}", 1, 'red')
                        raise ValueError(f"Imputation failed for {df_name} using {imp_name}. Remaining NaNs in columns: {nan_columns}")

                    df_output_dir = os.path.join(output_folder, imp_name)
                    os.makedirs(df_output_dir, exist_ok=True)
                    output_file = os.path.join(df_output_dir, f'{df_name}.csv')

                    if os.path.exists(output_file):
                        os.remove(output_file)
                        logger.log(f'Existing file {output_file} removed.', 3, 'yellow')

                    imputed_df.to_csv(output_file, index=False)
                    logger.log(f'Successfully imputed and saved {df_name} using {imp_name} to {output_file}', 2, 'green')

                    results[df_name]["imputed"][imp_name] = output_file

                    if clean_memory:
                        del imputed_chunks
                        del df
                        gc.collect()
                        logger.log(f'Original dataframe {df_name} deleted and memory cleaned.', 3)

                except Exception as e:
                    logger.log(f"Failed to impute {df_name} using {imp_name} due to {e}", 2, 'red')

                task_count += 1
                progress = (task_count / total_tasks) * 100
                logger.log(f'Progress: {task_count}/{total_tasks} ({progress:.2f}%)', 2)
                tqdm.write(f'Progress: {task_count}/{total_tasks} ({progress:.2f}%)')
        except Exception as e:
            logger.log(f"Failed to process {csv_file} due to {e}", 2, 'red')

    json_output_file = os.path.join(output_folder, f'imputation_results{"_test" if testing else ""}.json')
    with open(json_output_file, 'w') as json_file:
        json.dump(results, json_file, indent=4)
    logger.log(f'Successfully exported imputation results to {json_output_file}', 2, 'green')

    logger.export()

### Imputation method settings

In [ ]:
imputations = [
    ('median', SimpleImputer(strategy='median')),
    ('mean', SimpleImputer(strategy='mean')),
    ('most_frequent', SimpleImputer(strategy='most_frequent')),
    ('constant_0', SimpleImputer(strategy='constant', fill_value=0)),
    ('knn', KNNImputer(n_neighbors=5)),
    ('iterative', IterativeImputer()),
    ('ffill', lambda x: x.ffill()),  # Forward fill
    ('bfill', lambda x: x.bfill())   # Backward fill
]

### Imputation method call

In [ ]:
impute_and_save_dataframes_from_folder(
    input_folder=LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH,
    imputations=imputations,
    chunk_size=GENERAL_CHUNK_SIZE,
    output_folder=LOCAL_EXPORT_IMPUTATION_FOLDER_PATH,
    clean_memory=True,
    exclude_columns=["SK_ID_PREV", "SK_ID_CURR","SK_ID_BUREAU"],
    testing=TESTING_MODE,
    max_rows=TESTING_MODE_MAX_LINES,
    testing_data_sub_folder=TESTING_MODE_SUB_FOLDER_NAME,
    round_decimals=3,
    log_level=2,
    export_logs=True,
    log_folder_path=LOCAL_EXPORT_IMPUTATION_FOLDER_PATH)

In [ ]:
# Configure logging
logging.basicConfig(filename='data_processing.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s')

def validate_dataframes(original, imputed):
    # Ensure the dataframes have the same columns
    common_columns = original.columns.intersection(imputed.columns)
    return original[common_columns], imputed[common_columns]

def replace_missing_values(df, value=0):
    """
    Remplace les valeurs manquantes dans un DataFrame par une valeur donnée.

    :param df: DataFrame avec des valeurs manquantes.
    :param value: Valeur de remplacement (par défaut 0).
    :return: DataFrame avec les valeurs manquantes remplacées.
    """
    return df.fillna(value)

def check_alignment_and_calculate_rmse(original, imputed, mask, replacement_value=0):
    """
    Calcule le RMSE entre les valeurs originales et imputées, uniquement sur les valeurs qui étaient manquantes.

    :param original: DataFrame original.
    :param imputed: DataFrame imputé.
    :param mask: Masque indiquant les positions des valeurs manquantes initiales.
    :param replacement_value: Valeur de remplacement pour les valeurs manquantes dans le calcul (par défaut 0).
    :return: RMSE.
    """
    original_filled = replace_missing_values(original, replacement_value)
    diff = original_filled[mask] - imputed[mask]
    mse = (diff ** 2).mean().mean()
    rmse = np.sqrt(mse)
    return rmse

def scan_directory_for_imputations(input_folder):
    imputation_methods = {}
    for method in os.listdir(input_folder):
        method_path = os.path.join(input_folder, method)
        if os.path.isdir(method_path):
            csv_files = {os.path.splitext(file)[0]: os.path.join(method_path, file)
                         for file in os.listdir(method_path) if file.endswith('.csv')}
            imputation_methods[method] = csv_files
    return imputation_methods

def preprocess_dataframe(df):
    # Drop columns that are not numeric
    df = df.select_dtypes(include=[np.number])
    return df

def evaluate_imputation_methods(originals_folder, imputations_folder, output_folder, chunk_size=10000, testing=False, max_rows=200, testing_data_sub_folder='testing_data'):
    """
    Évalue les méthodes d'imputation sur plusieurs DataFrames originaux et leurs imputations correspondantes par chunks.

    :param originals_folder: Chemin du dossier contenant les DataFrames originaux non imputés.
    :param imputations_folder: Chemin du dossier contenant les DataFrames imputés par différentes méthodes.
    :param output_folder: Chemin du dossier où les résultats doivent être exportés.
    :param chunk_size: Taille des chunks pour le traitement des DataFrames.
    :param testing: Booléen indiquant si le mode test est activé pour traiter un échantillon réduit de données.
    :param max_rows: Nombre maximal de lignes à traiter en mode test.
    :param testing_data_sub_folder: Nom du sous-dossier pour les données de test en mode testing.
    :return: Dictionnaire contenant les scores RMSE moyens pour chaque méthode d'imputation pour chaque DataFrame original.
    """
    if testing:
        imputations_folder = os.path.join(imputations_folder, testing_data_sub_folder)
        output_folder = os.path.join(output_folder, testing_data_sub_folder)
        originals_folder = os.path.join(originals_folder, testing_data_sub_folder)

    imputation_methods = scan_directory_for_imputations(imputations_folder)
    results = {}

    if testing and not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for method, files in imputation_methods.items():
        for df_name, file_path in files.items():
            try:
                # Extraire le nom du fichier original en enlevant le suffixe d'imputation
                original_file_path = os.path.join(originals_folder, f"{df_name}.csv")

                if not os.path.exists(original_file_path):
                    logging.error(f"Original file {original_file_path} not found.")
                    continue

                original_df = pd.read_csv(original_file_path)
                imputed_df = pd.read_csv(file_path)

                original_df = preprocess_dataframe(original_df)
                imputed_df = preprocess_dataframe(imputed_df)

                if testing:
                    original_df = original_df.iloc[:max_rows]  # Take only the first max_rows for test mode
                    imputed_df = imputed_df.iloc[:max_rows]  # Take only the first max_rows for test mode

                # Créer un masque des valeurs manquantes initiales
                missing_mask = original_df.isnull()

                if df_name not in results:
                    results[df_name] = {"original": original_file_path, "imputed": {}}

                total_chunks = 0
                method_scores = []

                for chunk_start in tqdm(range(0, original_df.shape[0], chunk_size), desc=f'Evaluating {df_name} with {method}', leave=False):
                    chunk_end = min(chunk_start + chunk_size, original_df.shape[0])
                    original_chunk = original_df.iloc[chunk_start:chunk_end].copy()
                    imputed_chunk = imputed_df.iloc[chunk_start:chunk_end].copy()

                    numeric_cols = original_chunk.select_dtypes(include=[np.number]).columns

                    total_chunks += 1
                    original_chunk_cleaned, imputed_chunk_cleaned = validate_dataframes(original_chunk, imputed_chunk)

                    # Ajout de la vérification par assertion avant le calcul du RMSE
                    assert not imputed_chunk_cleaned.isnull().values.any(), f"NaN values found in the imputed chunk for {df_name} using method {method}"

                    if not original_chunk_cleaned.empty and not imputed_chunk_cleaned.empty:
                        rmse = check_alignment_and_calculate_rmse(original_chunk_cleaned, imputed_chunk_cleaned, missing_mask.iloc[chunk_start:chunk_end], replacement_value=0)
                        method_scores.append(rmse)

                if method_scores:
                    avg_rmse = np.mean(method_scores)
                    results[df_name]["imputed"][method] = avg_rmse

                if testing:
                    json_output_file = os.path.join(output_folder, 'imputation_results_test.json')
                else:
                    json_output_file = os.path.join(output_folder, 'imputation_results.json')

                with open(json_output_file, 'w') as json_file:
                    json.dump(results, json_file, indent=4)
                logging.info(f'Successfully exported imputation results to {json_output_file}')

            except Exception as e:
                logging.error(f"Failed to process {file_path} due to {e}")

    return results

def verify_imputed_files(imputations_folder):
    """
    Vérifie que les fichiers imputés ne contiennent plus de valeurs NaN ou nulles et génère une assertion en cas d'échec.

    :param imputations_folder: Chemin du dossier contenant les DataFrames imputés par différentes méthodes.
    :return: None
    """
    imputation_methods = scan_directory_for_imputations(imputations_folder)
    verification_results = {}

    for method, files in imputation_methods.items():
        method_results = {}
        for df_name, file_path in files.items():
            try:
                imputed_df = pd.read_csv(file_path)
                if imputed_df.isnull().values.any():
                    method_results[df_name] = False
                else:
                    method_results[df_name] = True
            except Exception as e:
                logging.error(f"Failed to verify {file_path} due to {e}")
                method_results[df_name] = False

        verification_results[method] = method_results

    # Affichage des résultats de la vérification
    for method, results in verification_results.items():
        print(f"Verification results for method {method}:")
        for df_name, result in results.items():
            status = "Passed" if result else "Failed"
            print(f" - {df_name}: {status}")
            assert result, f"Verification failed for {df_name} using method {method}. NaN or null values found."

    return verification_results


In [ ]:
evaluate_imputation_methods(
    originals_folder=LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH,
    imputations_folder=LOCAL_EXPORT_IMPUTATION_FOLDER_PATH,
    output_folder=LOCAL_EXPORT_IMPUTATION_CHECKS_FOLDER_PATH,
    chunk_size=GENERAL_CHUNK_SIZE,
    testing=TESTING_MODE,
    max_rows=TESTING_MODE_MAX_LINES,
    testing_data_sub_folder=TESTING_MODE_SUB_FOLDER_NAME)

In [ ]:
test2 = pd.read_csv(LOCAL_EXPORT_IMPUTATION_FOLDER_PATH+"/testing_data/bfill/application_train.csv")

In [ ]:
test2.head()

## Imputations Performances Evaluations

### Imputation Performances Evaluations Methods

#### Check align & Calculate RMSE method

In [ ]:
def check_alignment_and_calculate_rmse(original_df, imputed_df):
    """
    Vérifie l'alignement des valeurs imputées avec les valeurs originales et calcule le RMSE.

    :param original_df: DataFrame original avec des valeurs manquantes
    :param imputed_df: DataFrame avec les valeurs imputées
    :return: RMSE entre les valeurs originales et imputées
    """
    # Assurez-vous que les indices sont alignés
    if not original_df.index.equals(imputed_df.index):
        raise ValueError("Les indices des DataFrames original et imputé ne sont pas alignés.")

    # Sélectionner uniquement les colonnes numériques
    numeric_cols = original_df.select_dtypes(include=[np.number]).columns
    original_df_numeric = original_df[numeric_cols]
    imputed_df_numeric = imputed_df[numeric_cols]

    # Masquer les valeurs manquantes dans les données originales
    mask = original_df_numeric.isna()
    original_masked = original_df_numeric[mask]
    imputed_values = imputed_df_numeric[mask]

    # Vérifiez l'alignement des valeurs
    if not original_masked.index.equals(imputed_values.index):
        raise ValueError("Les indices des valeurs manquantes et imputées ne sont pas alignés.")
    if not all(original_masked.columns == imputed_values.columns):
        raise ValueError("Les colonnes des valeurs manquantes et imputées ne sont pas alignées.")

    # Convertir en arrays pour comparaison
    y_true = original_masked.values.flatten()
    y_pred = imputed_values.values.flatten()

    # Filtrer les NaN avant de calculer le RMSE
    non_nan_mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    y_true = y_true[non_nan_mask]
    y_pred = y_pred[non_nan_mask]

    if len(y_true) == 0 or len(y_pred) == 0:
        raise ValueError("Aucune donnée valide pour calculer le RMSE.")

    # Calculer le RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

#### Validate dataframes method

In [ ]:
def validate_dataframes(original_df, imputed_df):
    """
    Vérifie et nettoie les DataFrames avant l'évaluation.

    :param original_df: DataFrame original avec des valeurs manquantes
    :param imputed_df: DataFrame avec les valeurs imputées
    :return: DataFrames nettoyés et alignés
    """
    # Assurez-vous que les indices sont alignés
    if not original_df.index.equals(imputed_df.index):
        raise ValueError("Les indices des DataFrames original et imputé ne sont pas alignés.")

    # Sélectionner uniquement les colonnes numériques
    numeric_cols = original_df.select_dtypes(include=[np.number]).columns
    original_df_numeric = original_df[numeric_cols]
    imputed_df_numeric = imputed_df[numeric_cols]

    # Supprimer les colonnes où toutes les valeurs sont manquantes dans le DataFrame original
    valid_columns = original_df_numeric.columns[original_df_numeric.notna().any()].tolist()
    original_df_cleaned = original_df_numeric[valid_columns]
    imputed_df_cleaned = imputed_df_numeric[valid_columns]

    return original_df_cleaned, imputed_df_cleaned

#### Evaluate imputation method

In [ ]:
# Configure logging
logging.basicConfig(filename='data_processing.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s')

def validate_dataframes(original, imputed):
    # Ensure the dataframes have the same columns
    common_columns = original.columns.intersection(imputed.columns)
    return original[common_columns], imputed[common_columns]

def replace_missing_values(df, value=0):
    """
    Remplace les valeurs manquantes dans un DataFrame par une valeur donnée.

    :param df: DataFrame avec des valeurs manquantes.
    :param value: Valeur de remplacement (par défaut 0).
    :return: DataFrame avec les valeurs manquantes remplacées.
    """
    return df.fillna(value)

def check_alignment_and_calculate_rmse(original, imputed, mask, replacement_value=0):
    """
    Calcule le RMSE entre les valeurs originales et imputées, uniquement sur les valeurs qui étaient manquantes.

    :param original: DataFrame original.
    :param imputed: DataFrame imputé.
    :param mask: Masque indiquant les positions des valeurs manquantes initiales.
    :param replacement_value: Valeur de remplacement pour les valeurs manquantes dans le calcul (par défaut 0).
    :return: RMSE.
    """
    original_filled = replace_missing_values(original, replacement_value)
    diff = original_filled[mask] - imputed[mask]
    mse = (diff ** 2).mean().mean()
    rmse = np.sqrt(mse)
    return rmse

def scan_directory_for_imputations(input_folder):
    imputation_methods = {}
    for method in os.listdir(input_folder):
        method_path = os.path.join(input_folder, method)
        if os.path.isdir(method_path):
            csv_files = {os.path.splitext(file)[0]: os.path.join(method_path, file)
                         for file in os.listdir(method_path) if file.endswith('.csv')}
            imputation_methods[method] = csv_files
    return imputation_methods

def preprocess_dataframe(df):
    # Drop columns that are not numeric
    df = df.select_dtypes(include=[np.number])
    return df

def evaluate_imputation_methods(originals_folder, imputations_folder, output_folder, chunk_size=10000, testing=False, max_rows=200, testing_data_sub_folder='testing_data'):
    """
    Évalue les méthodes d'imputation sur plusieurs DataFrames originaux et leurs imputations correspondantes par chunks.

    :param originals_folder: Chemin du dossier contenant les DataFrames originaux non imputés.
    :param imputations_folder: Chemin du dossier contenant les DataFrames imputés par différentes méthodes.
    :param output_folder: Chemin du dossier où les résultats doivent être exportés.
    :param chunk_size: Taille des chunks pour le traitement des DataFrames.
    :param testing: Booléen indiquant si le mode test est activé pour traiter un échantillon réduit de données.
    :param max_rows: Nombre maximal de lignes à traiter en mode test.
    :param testing_data_sub_folder: Nom du sous-dossier pour les données de test en mode testing.
    :return: Dictionnaire contenant les scores RMSE moyens pour chaque méthode d'imputation pour chaque DataFrame original.
    """
    if testing:
        imputations_folder = os.path.join(imputations_folder, testing_data_sub_folder)
        output_folder = os.path.join(output_folder, testing_data_sub_folder)
        originals_folder = os.path.join(originals_folder, testing_data_sub_folder)

    imputation_methods = scan_directory_for_imputations(imputations_folder)
    results = {}

    if testing and not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for method, files in imputation_methods.items():
        for df_name, file_path in files.items():
            try:
                # Extraire le nom du fichier original en enlevant le suffixe d'imputation
                original_file_path = os.path.join(originals_folder, f"{df_name}.csv")

                if not os.path.exists(original_file_path):
                    logging.error(f"Original file {original_file_path} not found.")
                    continue

                original_df = pd.read_csv(original_file_path)
                imputed_df = pd.read_csv(file_path)

                original_df = preprocess_dataframe(original_df)
                imputed_df = preprocess_dataframe(imputed_df)

                if testing:
                    original_df = original_df.iloc[:max_rows]  # Take only the first max_rows for test mode
                    imputed_df = imputed_df.iloc[:max_rows]  # Take only the first max_rows for test mode

                # Créer un masque des valeurs manquantes initiales
                missing_mask = original_df.isnull()

                if df_name not in results:
                    results[df_name] = {"original": original_file_path, "imputed": {}}

                total_chunks = 0
                method_scores = []

                for chunk_start in tqdm(range(0, original_df.shape[0], chunk_size), desc=f'Evaluating {df_name} with {method}', leave=False):
                    chunk_end = min(chunk_start + chunk_size, original_df.shape[0])
                    original_chunk = original_df.iloc[chunk_start:chunk_end].copy()
                    imputed_chunk = imputed_df.iloc[chunk_start:chunk_end].copy()

                    numeric_cols = original_chunk.select_dtypes(include=[np.number]).columns

                    total_chunks += 1
                    original_chunk_cleaned, imputed_chunk_cleaned = validate_dataframes(original_chunk, imputed_chunk)
                    if not original_chunk_cleaned.empty and not imputed_chunk_cleaned.empty:
                        rmse = check_alignment_and_calculate_rmse(original_chunk_cleaned, imputed_chunk_cleaned, missing_mask.iloc[chunk_start:chunk_end], replacement_value=0)
                        method_scores.append(rmse)

                if method_scores:
                    avg_rmse = np.mean(method_scores)
                    results[df_name]["imputed"][method] = avg_rmse

                if testing:
                    json_output_file = os.path.join(output_folder, 'imputation_results_test.json')
                else:
                    json_output_file = os.path.join(output_folder, 'imputation_results.json')

                with open(json_output_file, 'w') as json_file:
                    json.dump(results, json_file, indent=4)
                logging.info(f'Successfully exported imputation results to {json_output_file}')

            except Exception as e:
                logging.error(f"Failed to process {file_path} due to {e}")

    return results

#### Plot result method

In [ ]:
def plot_results(results):
    """
    Affiche les résultats des scores RMSE moyens pour chaque méthode d'imputation pour chaque DataFrame original.

    :param results: Dictionnaire contenant les scores RMSE moyens pour chaque méthode d'imputation pour chaque DataFrame original.
    """
    for df_name, df_results in results.items():
        method_names = list(df_results['imputed'].keys())
        rmse_scores = list(df_results['imputed'].values())

        plt.figure(figsize=(10, 5))
        plt.bar(method_names, rmse_scores, color='skyblue')
        plt.xlabel('Méthodes d\'imputation')
        plt.ylabel('RMSE moyen')
        plt.title(f'Comparaison des performances des méthodes d\'imputation pour {df_name}')
        plt.xticks(rotation=45)
        plt.show()

### Call

In [ ]:
result = evaluate_imputation_methods(
      originals_folder=LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH,
        imputations_folder=LOCAL_EXPORT_IMPUTATION_FOLDER_PATH,
        output_folder=LOCAL_EXPORT_IMPUTATION_RESULTS_FOLDER_PATH,
        chunk_size=GENERAL_CHUNK_SIZE,
        testing=TESTING_MODE,
        max_rows=TESTING_MODE_MAX_LINES,
        testing_data_sub_folder=TESTING_MODE_SUB_FOLDER_NAME
    )

In [ ]:
def read_and_display_results(results_folder, testing=False, testing_data_sub_folder='testing_data'):
    """
    Lit et affiche les résultats des évaluations d'imputation.

    :param results_folder: Chemin du dossier contenant les fichiers de résultats JSON.
    :param testing: Booléen indiquant si le mode test est activé.
    :param testing_data_sub_folder: Nom du sous-dossier pour les résultats de test en mode testing.
    :return: DataFrame des résultats d'imputation.
    """
    # Déterminer le chemin du fichier de résultats
    if testing:
        results_file_path = os.path.join(results_folder, testing_data_sub_folder, 'imputation_results_test.json')
    else:
        results_file_path = os.path.join(results_folder, 'imputation_results.json')

    # Vérifier si le fichier de résultats existe
    if not os.path.exists(results_file_path):
        print(f"Le fichier de résultats {results_file_path} n'existe pas.")
        return

    # Lire les résultats du fichier JSON
    with open(results_file_path, 'r') as json_file:
        results = json.load(json_file)

    # Afficher les résultats de manière structurée
    imputation_results = []
    for df_name, df_results in results.items():
        original_file = df_results['original']
        for method, score in df_results['imputed'].items():
            imputation_results.append({
                'DataFrame': df_name,
                'Method': method,
                'RMSE': score,
                'Original File': original_file
            })

    # Convertir les résultats en DataFrame
    results_df = pd.DataFrame(imputation_results)

    # Afficher les résultats
    print("Résultats de l'évaluation des méthodes d'imputation :")
    print(results_df)

    # Afficher les résultats graphiquement
    plot_results(results)

    return results_df

In [ ]:
results_df = read_and_display_results(LOCAL_EXPORT_IMPUTATION_RESULTS_FOLDER_PATH, testing=True, testing_data_sub_folder=TESTING_MODE_SUB_FOLDER_NAME)

### Plot displaying

In [ ]:
def compare_original_and_imputed(originals_folder, imputations_folder, output_folder, num_lines=5, testing=False, testing_data_sub_folder='testing_data'):
    """
    Compare les valeurs originales et imputées pour les 5 premières lignes de chaque DataFrame.

    :param originals_folder: Chemin du dossier contenant les DataFrames originaux non imputés.
    :param imputations_folder: Chemin du dossier contenant les DataFrames imputés par différentes méthodes.
    :param output_folder: Chemin du dossier où les résultats doivent être exportés.
    :param num_lines: Nombre de lignes à comparer (par défaut 5).
    :param testing: Booléen indiquant si le mode test est activé.
    :param testing_data_sub_folder: Nom du sous-dossier pour les données de test en mode testing.
    """
    if testing:
        imputations_folder = os.path.join(imputations_folder, testing_data_sub_folder)
        originals_folder = os.path.join(originals_folder, testing_data_sub_folder)

    imputation_methods = scan_directory_for_imputations(imputations_folder)

    for method, files in imputation_methods.items():
        for df_name, file_path in files.items():
            try:
                # Extraire le nom du fichier original en enlevant le suffixe d'imputation
                # df_name = '_'.join(df_name_with_imputation.split('_')[:-2])
                original_file_path = os.path.join(originals_folder, f"{df_name}.csv")

                if not os.path.exists(original_file_path):
                    logging.error(f"Original file {original_file_path} not found.")
                    continue

                original_df = pd.read_csv(original_file_path)
                imputed_df = pd.read_csv(file_path)

                original_df = preprocess_dataframe(original_df)
                imputed_df = preprocess_dataframe(imputed_df)

                # Prendre les num_lines premières lignes
                original_sample = original_df.head(num_lines)
                imputed_sample = imputed_df.head(num_lines)

                # Créer un DataFrame pour la comparaison
                comparison_df = pd.DataFrame()

                for i in range(num_lines):
                    original_row = original_sample.iloc[i].rename(f"Original_Ligne_{i+1}")
                    imputed_row = imputed_sample.iloc[i].rename(f"Impute_Ligne_{i+1}")
                    comparison_df = pd.concat([comparison_df, original_row, imputed_row], axis=1)

                print(f"Comparaison des premières lignes pour {df_name} avec la méthode {method}:")
                display(comparison_df.T)

            except Exception as e:
                logging.error(f"Failed to process {file_path} due to {e}")

# Fonction d'aide pour afficher les DataFrames de manière plus claire
def display(df):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    print(df)

In [ ]:
compare_original_and_imputed( originals_folder=LOCAL_EXPORT_COLUMNS_TYPES_CHECKED_FOLDER_PATH,
        imputations_folder=LOCAL_EXPORT_IMPUTATION_FOLDER_PATH,
        output_folder=LOCAL_EXPORT_IMPUTATION_RESULTS_FOLDER_PATH,
        testing=TESTING_MODE,
        testing_data_sub_folder=TESTING_MODE_SUB_FOLDER_NAME)

## Outlier handling

### Interquartile Range

In [ ]:
def identify_outliers_iqr(chunk):
    outliers_dict = {}
    for column in chunk.select_dtypes(include=[np.number]).columns:
        Q1 = chunk[column].quantile(0.25)
        Q3 = chunk[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = chunk[(chunk[column] < lower_bound) | (chunk[column] > upper_bound)]
        outliers_dict[column] = outliers
    return outliers_dict

### Z-score

In [ ]:
def identify_outliers_z_score(chunk):
    outliers_dict = {}
    for column in chunk.select_dtypes(include=[np.number]).columns:
        mean = chunk[column].mean()
        std = chunk[column].std()
        chunk['z_score'] = (chunk[column] - mean) / std

        outliers = chunk[(chunk['z_score'] > 3) | (chunk['z_score'] < -3)]
        chunk.drop(columns=['z_score'], inplace=True)
        outliers_dict[column] = outliers
    return outliers_dict

### Isolation Forest

In [ ]:
def identify_outliers_isolation_forest(chunk):
    outliers_dict = {}
    for column in chunk.select_dtypes(include=[np.number]).columns:
        model = IsolationForest(contamination=0.05)
        chunk['anomaly'] = model.fit_predict(chunk[[column]])
        outliers = chunk[chunk['anomaly'] == -1]
        chunk.drop(columns=['anomaly'], inplace=True)
        outliers_dict[column] = outliers
    return outliers_dict

### DBScan

In [ ]:
def identify_outliers_dbscan(chunk):
    outliers_dict = {}
    for column in chunk.select_dtypes(include=[np.number]).columns:
        model = DBSCAN(eps=3, min_samples=2)
        chunk['cluster'] = model.fit_predict(chunk[[column]])
        outliers = chunk[chunk['cluster'] == -1]
        chunk.drop(columns=['cluster'], inplace=True)
        outliers_dict[column] = outliers
    return outliers_dict

### LOF

In [ ]:
def identify_outliers_lof(chunk):
    outliers_dict = {}
    for column in chunk.select_dtypes(include=[np.number]).columns:
        model = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
        chunk['lof'] = model.fit_predict(chunk[[column]])
        outliers = chunk[chunk['lof'] == -1]
        outliers_dict[column] = outliers
    return outliers_dict

### MAD

In [ ]:
def identify_outliers_mad(chunk):
    outliers_dict = {}
    for column in chunk.select_dtypes(include=[np.number]).columns:
        median = chunk[column].median()
        mad = (chunk[column] - median).abs().median()
        threshold = 3 * mad
        lower_bound = median - threshold
        upper_bound = median + threshold

        outliers = chunk[(chunk[column] < lower_bound) | (chunk[column] > upper_bound)]
        outliers_dict[column] = outliers
    return outliers_dict

### Plot box

In [ ]:
def plot_box(chunk, output_dir, file_name, chunk_num):
    for column in chunk.select_dtypes(include=[np.number]).columns:
        plt.figure(figsize=(10, 5))
        plt.boxplot(chunk[column])
        plt.title(f'Box plot of {column} - {file_name} - Chunk {chunk_num}')
        plt.ylabel(column)
        plt.savefig(os.path.join(output_dir, f'boxplot_{file_name}_chunk{chunk_num}_{column}.png'))
        plt.close()

### Remove method

In [ ]:
def remove_outliers(chunk, outliers_dict):
    for column, outliers in outliers_dict.items():
        if not outliers.empty:
            chunk = chunk[~chunk.index.isin(outliers.index)]
    return chunk

### Processing

In [ ]:
class Logger:
    def __init__(self, level=4, export_logs=False, log_folder_path='logs'):
        self.log_level = level
        self.export_logs = export_logs
        self.log_folder_path = log_folder_path
        self.log_messages = []

    def log(self, message, level=4, color=None):
        if level <= self.log_level:
            if color:
                print(colored(message, color))
            else:
                print(message)
            self.log_messages.append(message)

    def export(self, log_file_name='logs.txt'):
        if self.export_logs:
            log_file_path = os.path.join(self.log_folder_path, log_file_name)
            os.makedirs(self.log_folder_path, exist_ok=True)
            with open(log_file_path, 'w') as log_file:
                for message in self.log_messages:
                    log_file.write(message + '\n')
            print(colored(f'Logs exported to {log_file_path}', 'green'))

In [ ]:
def outliers_cleaning(input_folder, output_folder, chunk_size=100000, testing=False, max_rows=200, testing_data_sub_folder='testing_data', logger=None):
    if logger is None:
        logger = Logger()

    if testing:
        output_folder = os.path.join(output_folder, testing_data_sub_folder)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    results = {}

    files_to_process = []
    for root, dirs, files in os.walk(input_folder):
        if testing and testing_data_sub_folder not in root:
            continue
        for file in files:
            if file.endswith('.csv'):
                files_to_process.append((root, file))

    for root, file in tqdm(files_to_process, desc="Processing files"):
        file_path = os.path.join(root, file)
        file_name = os.path.basename(file_path)
        chunk_num = 0

        logger.log(f'Starting processing of {file_path}', level=3)

        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            if testing and chunk_num * chunk_size >= max_rows:
                break

            chunk_num += 1
            logger.log(f'Processing chunk {chunk_num} of {file_name}', level=2)

            # Dictionnaire pour stocker les chunks nettoyés par méthode
            cleaned_chunks = {
                'IQR': chunk.copy(),
                'Z-Score': chunk.copy(),
                'Isolation Forest': chunk.copy(),
                'DBSCAN': chunk.copy(),
                'LOF': chunk.copy(),
                'MAD': chunk.copy()
            }

            # Identification et nettoyage des outliers pour chaque méthode
            outliers_methods = {
                'IQR': identify_outliers_iqr(chunk),
                'Z-Score': identify_outliers_z_score(chunk),
                'Isolation Forest': identify_outliers_isolation_forest(chunk),
                'DBSCAN': identify_outliers_dbscan(chunk),
                'LOF': identify_outliers_lof(chunk),
                'MAD': identify_outliers_mad(chunk)
            }

            for method, outliers in outliers_methods.items():
                all_outliers = pd.concat([df for df in [outliers.get(column) for column in outliers] if df is not None]).drop_duplicates()
                cleaned_chunks[method] = chunk[~chunk.index.isin(all_outliers.index)]

                relative_path = os.path.relpath(root, input_folder)
                if testing and relative_path.startswith(testing_data_sub_folder):
                    relative_path = relative_path[len(testing_data_sub_folder):].lstrip(os.sep)

                # Dossiers de sortie pour les données nettoyées et les outliers
                method_output_dir = os.path.join(output_folder, relative_path, method)
                outliers_output_dir = os.path.join(method_output_dir, 'outliers')

                if not os.path.exists(method_output_dir):
                    os.makedirs(method_output_dir)
                if not os.path.exists(outliers_output_dir):
                    os.makedirs(outliers_output_dir)

                # Export du chunk nettoyé
                output_csv_path = os.path.join(method_output_dir, f'{file_name}')
                cleaned_chunks[method].to_csv(output_csv_path, index=False)

                # Export des outliers
                outliers_csv_path = os.path.join(outliers_output_dir, f'{file_name}')
                all_outliers.to_csv(outliers_csv_path, index=False)

                if file_name not in results:
                    results[file_name] = {
                        'IQR': 0,
                        'Z-Score': 0,
                        'Isolation Forest': 0,
                        'DBSCAN': 0,
                        'LOF': 0,
                        'MAD': 0
                    }

                results[file_name][method] += len(all_outliers)

            logger.log(f'Finished processing chunk {chunk_num} of {file_name}', level=2)

            del chunk
            del cleaned_chunks
            del outliers_methods

    return results

In [ ]:
app_train_validated = pd.read_csv('/content/exports/imputed_without_outliers/testing_data/bfill/IQR/application_train.csv')
app_train_outliers = pd.read_csv('/content/exports/imputed_without_outliers/testing_data/bfill/IQR/application_train.csv')

In [ ]:
app_train_validated.head()

In [ ]:
app_train_outliers.head()

### Call

In [ ]:
outliers_results = outliers_cleaning(
  input_folder=LOCAL_EXPORT_IMPUTATION_FOLDER_PATH,
  output_folder=LOCAL_EXPORT_IMPUTED_WITHOUT_OUTLIERS_FOLDER_PATH,
  chunk_size=GENERAL_CHUNK_SIZE,
  testing=TESTING_MODE,
  max_rows=TESTING_MODE_MAX_LINES,
  testing_data_sub_folder=TESTING_MODE_SUB_FOLDER_NAME)
outliers_results

## Duplicate managment

### Duplicate managment method

In [ ]:
for root, _, files in os.walk(os.path.join(LOCAL_EXPORT_IMPUTED_WITHOUT_OUTLIERS_FOLDER_PATH, 'testing_data')):
  print(root)
  # print(_)
  print(files)

In [ ]:
def without_duplicates(base_dir, taille_chunk, dossier_export, mode_test=False, max_rows=1000, testing_data_sub_folder_name='testing_data'):
    fichiers = {}
    base_search_dir = os.path.join(base_dir, testing_data_sub_folder_name) if mode_test else base_dir

    for root, _, files in os.walk(base_search_dir):
        for file in files:
            if file.endswith('.csv') and "/outliers" not in root:  # Ignorer les fichiers outliers
                chemin_complet = os.path.join(root, file)
                cle = os.path.relpath(root, base_search_dir)
                if cle not in fichiers:
                    fichiers[cle] = []
                fichiers[cle].append(chemin_complet)
                logging.info(f'Fichier ajouté: {chemin_complet} sous clé: {cle}')

    # Log pour vérifier les fichiers trouvés
    logging.info(f'Fichiers trouvés: {fichiers.items()}')

    try:
        for cle, chemins in fichiers.items():
            df_complet = pd.DataFrame()
            unique_rows = set()
            for fichier in chemins:
                try:
                    logging.info(f'Traitement du fichier: {fichier}')
                    reader = pd.read_csv(fichier, chunksize=taille_chunk)
                    total_rows = sum(1 for row in open(fichier)) - 1  # Nombre total de lignes (moins l'en-tête)
                    pbar = tqdm(total=total_rows, desc=f'Traitement de {fichier}')

                    for chunk in reader:
                        chunk_tuples = [tuple(row) for row in chunk.to_numpy()]
                        new_rows = [row for row, row_tuple in zip(chunk.values, chunk_tuples) if row_tuple not in unique_rows]
                        unique_rows.update(chunk_tuples)
                        if new_rows:
                            df_complet = pd.concat([df_complet, pd.DataFrame(new_rows, columns=chunk.columns)], ignore_index=True)
                        pbar.update(len(chunk))
                        if mode_test and len(df_complet) >= max_rows:
                            break

                    pbar.close()

                except Exception as e:
                    logging.error(f'Erreur lors du traitement de {fichier}: {str(e)}')
                    if 'pbar' in locals():
                        pbar.close()

            dossier_export_final = os.path.join(dossier_export, testing_data_sub_folder_name, cle) if mode_test else os.path.join(dossier_export, cle)

            if not os.path.exists(dossier_export_final):
                os.makedirs(dossier_export_final)

            nom_fichier_export = os.path.join(dossier_export_final, f"{os.path.basename(fichier)}")
            df_complet.to_csv(nom_fichier_export, index=False)
            logging.info(f'Traitement de {cle} terminé avec succès. Fichier exporté : {nom_fichier_export}')

        print("Traitement des fichiers et exportation terminés.")

    except Exception as e:
        logging.critical(f'Erreur critique: {str(e)}')

### Call

In [ ]:
without_duplicates(
  base_dir=LOCAL_EXPORT_IMPUTED_WITHOUT_OUTLIERS_FOLDER_PATH,
  taille_chunk=GENERAL_CHUNK_SIZE,
  dossier_export=LOCAL_EXPORT_IMPUTED_WITHOUT_DUPLICATES_FOLDER_PATH,
  mode_test=TESTING_MODE,
  max_rows=TESTING_MODE_MAX_LINES,
  testing_data_sub_folder_name=TESTING_MODE_SUB_FOLDER_NAME)

## Feature Engineering

### Method

In [ ]:
logging.basicConfig(level=logging.INFO)

def feature_engineering(df):
    # Conversion des Jours en Années
    if 'DAYS_BIRTH' in df.columns:
        df['YEARS_BIRTH'] = df['DAYS_BIRTH'] / -365
    if 'DAYS_EMPLOYED' in df.columns:
        df['YEARS_EMPLOYED'] = df['DAYS_EMPLOYED'] / -365
    if 'DAYS_ID_PUBLISH' in df.columns:
        df['YEARS_ID_PUBLISH'] = df['DAYS_ID_PUBLISH'] / -365
    if 'DAYS_LAST_PHONE_CHANGE' in df.columns:
        df['YEARS_LAST_PHONE_CHANGE'] = df['DAYS_LAST_PHONE_CHANGE'] / -365

    # Interaction entre Âge et Durée de l'Emploi
    if 'YEARS_EMPLOYED' in df.columns and 'YEARS_BIRTH' in df.columns:
        df['AGE_EMPLOYED_RATIO'] = df['YEARS_EMPLOYED'] / df['YEARS_BIRTH']

    # Ratios Financiers
    if 'AMT_CREDIT' in df.columns and 'AMT_INCOME_TOTAL' in df.columns:
        df['CREDIT_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

    # Durée des Crédits Restants
    if 'DAYS_CREDIT_ENDDATE' in df.columns:
        df['CREDIT_DURATION'] = df['DAYS_CREDIT_ENDDATE'] / -365
    if 'DAYS_CREDIT_UPDATE' in df.columns:
        df['CREDIT_UPDATE_DURATION'] = df['DAYS_CREDIT_UPDATE'] / -365

    # Historique de Demandes de Crédit
    credit_request_columns = ['AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
                              'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
                              'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
    if any(col in df.columns for col in credit_request_columns):
        df['TOTAL_CREDIT_BUREAU_REQUESTS'] = df[credit_request_columns].sum(axis=1)

    # Indicateurs Basés sur la Connaissance du Domaine
    if 'CNT_CHILDREN' in df.columns and 'CNT_FAM_MEMBERS' in df.columns:
        df['CHILDREN_RATIO'] = df['CNT_CHILDREN'] / (df['CNT_FAM_MEMBERS'] + 1)
    if 'AMT_INCOME_TOTAL' in df.columns and 'CNT_FAM_MEMBERS' in df.columns:
        df['INCOME_PER_FAMILY_MEMBER'] = df['AMT_INCOME_TOTAL'] / (df['CNT_FAM_MEMBERS'] + 1)

    return df

def process_in_chunks(input_file, output_file, chunk_size=10000):
    columns_to_check = [
        'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE',
        'YEARS_EMPLOYED', 'YEARS_BIRTH', 'AMT_CREDIT', 'AMT_INCOME_TOTAL',
        'DAYS_CREDIT_ENDDATE', 'DAYS_CREDIT_UPDATE', 'CNT_CHILDREN', 'CNT_FAM_MEMBERS',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_WEEK',
        'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR'
    ]

    column_exists = False
    chunks = []

    # Read in chunks, apply feature engineering if applicable
    for chunk in pd.read_csv(input_file, chunksize=chunk_size):
        if any(col in chunk.columns for col in columns_to_check):
            column_exists = True
            chunk = feature_engineering(chunk)
        chunks.append(chunk)

    # Concatenate chunks and write to output
    df_full = pd.concat(chunks, ignore_index=True)
    df_full.to_csv(output_file, index=False)

    # If no relevant columns exist, copy the original file to ensure no data is lost
    if not column_exists:
        df_full.to_csv(output_file, index=False)

def process_feature_engineering_to_all_files(base_dir, output_dir, chunk_size=10000, mode_test=False, testing_data_sub_folder_name='testing_data'):
    base_search_dir = os.path.join(base_dir, testing_data_sub_folder_name) if mode_test else base_dir

    all_files = []
    for root, _, files in os.walk(base_search_dir):
        for file in files:
            chemin_complet = os.path.join(root, file)
            if file.endswith('.csv') and "/outliers" not in chemin_complet:  # Ignorer les fichiers outliers
                cle = os.path.relpath(root, base_search_dir)
                output_sub_dir = os.path.join(output_dir, testing_data_sub_folder_name, cle) if mode_test else os.path.join(output_dir, cle)
                input_file = chemin_complet
                output_file = os.path.join(output_sub_dir, f"{os.path.splitext(file)[0]}.csv")
                all_files.append((input_file, output_file, output_sub_dir))

    with tqdm(total=len(all_files), desc="Traitement des fichiers") as pbar:
        for input_file, output_file, output_sub_dir in all_files:
            if not os.path.exists(output_sub_dir):
                os.makedirs(output_sub_dir)
            logging.info(f'Traitement du fichier: {input_file}')
            process_in_chunks(input_file, output_file, chunk_size)
            logging.info(f'Traitement de {input_file} terminé avec succès. Fichier exporté : {output_file}')
            pbar.update(1)

    print("Traitement des fichiers et exportation terminés.")


### Call

In [ ]:
process_feature_engineering_to_all_files(base_dir=LOCAL_EXPORT_IMPUTED_WITHOUT_OUTLIERS_FOLDER_PATH,
                  output_dir=LOCAL_EXPORT_FEATURE_ENGINEERED_FOLDER_PATH,
                  chunk_size=GENERAL_CHUNK_SIZE,
                  mode_test=TESTING_MODE,
                  testing_data_sub_folder_name=TESTING_MODE_SUB_FOLDER_NAME)

In [ ]:
pd.read_csv('/content/exports/imputed_without_outliers/testing_data/bfill/LOF/application_train.csv')

In [ ]:
pd.read_csv('/content/exports/imputed_without_duplicates/testing_data/bfill/DBSCAN/application_train.csv')

In [ ]:
pd.read_csv('/content/exports/feature_engineered/testing_data/bfill/LOF/application_train.csv')

## Merge

In [ ]:
def aggregate_numerical(dataframe, group_by_column):
    """
    Aggregates numerical columns in the dataframe by calculating the mean, grouped by the specified column.

    Args:
    dataframe (pd.DataFrame): The input dataframe containing numerical columns to be aggregated.
    group_by_column (str): The column name to group by.

    Returns:
    pd.DataFrame: A new dataframe with the mean of numerical columns grouped by the specified column.
    """
    # Select numerical columns, excluding the group_by_column
    numerical_columns = dataframe.select_dtypes(include=['number']).columns
    numerical_columns = [col for col in numerical_columns if col != group_by_column]
    return dataframe.groupby(group_by_column)[numerical_columns].mean().reset_index()

def merge_data(application_train, application_test, bureau, bureau_balance, previous_application, pos_cash_balance, installments_payments, credit_card_balance):
    """
    Merges multiple dataframes into a single training and testing dataframe.

    Args:
    application_train (pd.DataFrame): Training application dataframe.
    application_test (pd.DataFrame): Testing application dataframe.
    bureau (pd.DataFrame): Bureau dataframe.
    bureau_balance (pd.DataFrame): Bureau balance dataframe.
    previous_application (pd.DataFrame): Previous application dataframe.
    pos_cash_balance (pd.DataFrame): POS cash balance dataframe.
    installments_payments (pd.DataFrame): Installments payments dataframe.
    credit_card_balance (pd.DataFrame): Credit card balance dataframe.

    Returns:
    tuple: A tuple containing merged training dataframe, merged testing dataframe, and aggregated previous application dataframe.
    """
    # Aggregate and merge bureau_balance with bureau
    bureau_balance_agg = aggregate_numerical(bureau_balance, 'SK_ID_BUREAU')
    bureau_balance_agg = bureau_balance_agg.add_suffix('_bureau_balance')
    bureau_balance_agg.rename(columns={'SK_ID_BUREAU_bureau_balance': 'SK_ID_BUREAU'}, inplace=True)
    bureau = bureau.merge(bureau_balance_agg, on='SK_ID_BUREAU', how='left')

    # Ensure SK_ID_CURR is present before merging
    if 'SK_ID_CURR' not in application_train.columns or 'SK_ID_CURR' not in bureau.columns:
        raise KeyError("SK_ID_CURR column is missing in one of the dataframes.")

    # Merge application_train and application_test with bureau data
    train_merged = application_train.merge(bureau, on='SK_ID_CURR', how='left', suffixes=('', '_bureau'))
    test_merged = application_test.merge(bureau, on='SK_ID_CURR', how='left', suffixes=('', '_bureau'))

    # Aggregate other balance and payment data by SK_ID_PREV
    pos_cash_balance_agg = aggregate_numerical(pos_cash_balance, 'SK_ID_PREV')
    installments_payments_agg = aggregate_numerical(installments_payments, 'SK_ID_PREV')
    credit_card_balance_agg = aggregate_numerical(credit_card_balance, 'SK_ID_PREV')

    # Merge aggregated data with previous_application
    previous_application = previous_application.merge(pos_cash_balance_agg, on='SK_ID_PREV', how='left', suffixes=('', '_pos_cash'))
    previous_application = previous_application.merge(installments_payments_agg, on='SK_ID_PREV', how='left', suffixes=('', '_installments'))
    previous_application = previous_application.merge(credit_card_balance_agg, on='SK_ID_PREV', how='left', suffixes=('', '_credit_card'))

    # Aggregate previous_application data by SK_ID_CURR
    previous_application_agg = aggregate_numerical(previous_application, 'SK_ID_CURR')

    # Merge aggregated previous_application data with train and test data
    train_merged = train_merged.merge(previous_application_agg, on='SK_ID_CURR', how='left', suffixes=('', '_prev_app'))
    test_merged = test_merged.merge(previous_application_agg, on='SK_ID_CURR', how='left', suffixes=('', '_prev_app'))

    return train_merged, test_merged, previous_application_agg

def export_to_csv_in_chunks(dataframe, filepath, chunksize):
    """
    Exports a dataframe to a CSV file in chunks to manage memory usage.

    Args:
    dataframe (pd.DataFrame): The dataframe to be exported.
    filepath (str): The path where the CSV file will be saved.
    chunksize (int): The number of rows per chunk to write at a time.
    """
    dataframe.to_csv(filepath, index=False, chunksize=chunksize)

def merge_and_export(input_base_path, output_base_path, testing=False, max_rows=None, chunk_size=10000, testing_data_sub_folder_name='test'):
    """
    Merges multiple dataframes from the specified input path and exports the merged dataframes to the specified output path.

    Args:
    input_base_path (str): The base path where the input CSV files are located.
    output_base_path (str): The base path where the output CSV files will be saved.
    testing (bool): Whether to activate testing mode (limit the number of rows and modify output paths).
    max_rows (int): The maximum number of rows to read from each input file (only applicable in testing mode).
    chunk_size (int): The number of rows per chunk to write at a time.
    testing_data_sub_folder_name (str): The sub-folder name to use in testing mode for saving output files.
    """
    # Walk through the input directory to process CSV files
    for root, dirs, files in tqdm(os.walk(input_base_path), desc="Processing directories"):
        if "/outliers" not in root:
            dataframes = {}
            # Read each CSV file and store in the dataframes dictionary
            for file in files:
                if file.endswith('.csv'):
                    filepath = os.path.join(root, file)
                    df = pd.read_csv(filepath)
                    if testing and max_rows:
                        df = df.head(max_rows)
                    if 'bureau_balance' in file:
                        dataframes['bureau_balance'] = df
                    elif 'bureau' in file:
                        dataframes['bureau'] = df
                    elif 'POS_CASH_balance' in file:
                        dataframes['POS_CASH_balance'] = df
                    elif 'previous_application' in file:
                        dataframes['previous_application'] = df
                    elif 'credit_card_balance' in file:
                        dataframes['credit_card_balance'] = df
                    elif 'installments_payments' in file:
                        dataframes['installments_payments'] = df
                    elif 'application_train' in file:
                        dataframes['application_train'] = df
                    elif 'application_test' in file:
                        dataframes['application_test'] = df

            # Check if all required dataframes are present
            required_keys = ['bureau', 'bureau_balance', 'POS_CASH_balance', 'previous_application', 'credit_card_balance', 'installments_payments', 'application_train', 'application_test']
            if not all(key in dataframes for key in required_keys):
                continue

            bureau = dataframes['bureau']
            bureau_balance = dataframes['bureau_balance']
            POS_CASH_balance = dataframes['POS_CASH_balance']
            previous_application = dataframes['previous_application']
            credit_card_balance = dataframes['credit_card_balance']
            installments_payments = dataframes['installments_payments']
            app_train = dataframes['application_train']
            app_test = dataframes['application_test']

            # Merge dataframes
            application_train_merged, application_test_merged, previous_application_agg = merge_data(app_train, app_test, bureau, bureau_balance, previous_application, POS_CASH_balance, installments_payments, credit_card_balance)

            # Extract methods for naming the output files
            imputation_method = root.split('/')[-2]
            outlier_method = root.split('/')[-1]
            # encoded_method = root.split('/')[-2]

            # Modify path logic to avoid additional "test" folder
            if testing:
                print(f"output_base_path={output_base_path}")
                print(f"testing_data_sub_folder_name={testing_data_sub_folder_name}")
                print(f"imputation_method={imputation_method}")
                print(f"outlier_method={outlier_method}")

                output_train_path = os.path.join(output_base_path, testing_data_sub_folder_name, imputation_method, outlier_method, 'application_train.csv')
                output_test_path = os.path.join(output_base_path, testing_data_sub_folder_name, imputation_method, outlier_method, 'application_test.csv')
            else:
                output_train_path = os.path.join(output_base_path, imputation_method, outlier_method, 'application_train.csv')
                output_test_path = os.path.join(output_base_path, imputation_method, outlier_method, 'application_test.csv')

            # Create directories if they do not exist
            os.makedirs(os.path.dirname(output_train_path), exist_ok=True)
            # Export merged training data
            export_to_csv_in_chunks(application_train_merged, output_train_path, chunk_size)

            # Clean up to free memory
            del app_train, application_train_merged
            gc.collect()

            # Merge testing data in chunks
            application_test_merged = app_test.merge(previous_application_agg, on='SK_ID_CURR', how='left', suffixes=('', '_prev_app'))

            # Create directories if they do not exist
            os.makedirs(os.path.dirname(output_test_path), exist_ok=True)
            # Export merged testing data
            export_to_csv_in_chunks(application_test_merged, output_test_path, chunk_size)

            # Clean up to free memory
            del app_test, application_test_merged
            gc.collect()

In [ ]:
dbcscan_bureau = pd.read_csv("/content/exports/imputed_without_outliers/testing_data/bfill/DBSCAN/bureau.csv")
application_test = pd.read_csv("/content/exports/imputed_without_outliers/testing_data/bfill/DBSCAN/application_test_processed.csv")
application_train = pd.read_csv("/content/exports/imputed_without_outliers/testing_data/bfill/DBSCAN/application_train_processed.csv")
credit_card_balance = pd.read_csv("/content/exports/imputed_without_outliers/testing_data/bfill/DBSCAN/credit_card_balance_processed.csv")


In [ ]:
dbcscan_bureau.head()


In [ ]:
application_test.head()


In [ ]:
application_train.head()
credit_card_balance.head()

### Call

In [ ]:
# Example usage:
merge_and_export(input_base_path=LOCAL_EXPORT_FEATURE_ENGINEERED_FOLDER_PATH, output_base_path=LOCAL_EXPORT_MERGED_DATAFRAMES_FOLDER_PATH, testing=TESTING_MODE, max_rows=TESTING_MODE_MAX_LINES, chunk_size=GENERAL_CHUNK_SIZE, testing_data_sub_folder_name=TESTING_MODE_SUB_FOLDER_NAME)

## Label Encoding and One-Hot Encoding

Let's implement the policy described above: for any categorical variable (`dtype == object`) with 2 unique categories, we will use label encoding, and for any categorical variable with more than 2 unique categories, we will use one-hot encoding.

For label encoding, we use the Scikit-Learn `LabelEncoder` and for one-hot encoding, the pandas `get_dummies(df)` function.

In [ ]:
def encode_and_align_chunked(input_dir, output_dir, target_columns=None, encoding_methods=['onehot'], chunk_size=10000, test_mode=False, max_rows=10000, testing_sub_folder_name="test_output"):
    """
    Processes and encodes chunks of CSV files, aligning training and testing data and applying specified encoding methods.

    Args:
    input_dir (str): The directory containing the input CSV files.
    output_dir (str): The directory where the processed CSV files will be saved.
    target_columns (list): List of target columns for encoding methods that require targets.
    encoding_methods (list): List of encoding methods to apply (e.g., ['onehot', 'label']).
    chunk_size (int): The number of rows per chunk to process at a time.
    test_mode (bool): Whether to limit the number of rows for testing purposes.
    max_rows (int): The maximum number of rows to process when test_mode is True.
    testing_sub_folder_name (str): The sub-folder name for saving outputs in testing mode.

    Returns:
    None
    """
    def process_file(train_file, test_file, output_train_file, output_test_file, encoding_method, pbar):
        """
        Processes a pair of training and testing files, applying encoding and alignment.

        Args:
        train_file (str): Path to the training CSV file.
        test_file (str): Path to the testing CSV file.
        output_train_file (str): Path to save the processed training CSV file.
        output_test_file (str): Path to save the processed testing CSV file.
        encoding_method (str): The encoding method to apply.
        pbar (tqdm): Progress bar instance for tracking progress.

        Returns:
        None
        """
        train_header_written = False
        test_header_written = False

        # Initialize encoders
        le = LabelEncoder()
        oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        be = ce.BinaryEncoder()
        he = ce.HashingEncoder()
        loo = ce.LeaveOneOutEncoder()
        woe = ce.WOEEncoder()
        pe = ce.PolynomialEncoder()
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

        # Dictionary to hold target labels if target columns are specified
        if target_columns:
            train_labels_list = {target: [] for target in target_columns}
            print(f"Dictionary to hold target labels initialized: {train_labels_list}")

        try:
            chunk_train_iterator = pd.read_csv(train_file, chunksize=chunk_size, on_bad_lines='skip')
            chunk_test_iterator = pd.read_csv(test_file, chunksize=chunk_size, on_bad_lines='skip')

            rows_processed = 0

            for chunk_train, chunk_test in zip(chunk_train_iterator, chunk_test_iterator):
                if test_mode and rows_processed >= max_rows:
                    break

                if chunk_train.empty or chunk_test.empty:
                    continue

                # Drop columns with all NaN values
                chunk_train.dropna(axis=1, how='all', inplace=True)
                chunk_test.dropna(axis=1, how='all', inplace=True)

                # Ensure both train and test chunks have common columns, excluding target columns from alignment
                common_columns = list(set(chunk_train.columns).intersection(set(chunk_test.columns)))

                if not common_columns:
                    continue

                chunk_train_common = chunk_train[common_columns]
                chunk_test_common = chunk_test[common_columns]

                # Print column names to debug
                print(f"Columns in training chunk from file {train_file}: {chunk_train.columns.tolist()}")
                print(f"Columns in testing chunk from file {test_file}: {chunk_test.columns.tolist()}")

                # Verify presence of target columns
                if target_columns:
                    for target in target_columns:
                        if target not in chunk_train.columns:
                            raise ValueError(f"Target column '{target}' not found in training data from file: {train_file}")
                    print(f"Target columns verified: {target_columns}")
                    for target in target_columns:
                        print(f"First values of target column '{target}' in training data: {chunk_train[target].head().values}")

                # Apply encoding methods
                if encoding_method == 'label':
                    for col in chunk_train_common.columns:
                        if chunk_train_common[col].dtype == 'object' or chunk_train_common[col].dtype.name == 'category':
                            le.fit(chunk_train_common[col])
                            chunk_train_common[col] = le.transform(chunk_train_common[col])
                            chunk_test_common[col] = chunk_test_common[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
                            le.classes_ = np.append(le.classes_, '<unknown>')
                            chunk_test_common[col] = le.transform(chunk_test_common[col])

                elif encoding_method == 'onehot':
                    categorical_columns = chunk_train_common.select_dtypes(include=['object']).columns
                    if len(categorical_columns) > 0:
                        ohe.fit(chunk_train_common[categorical_columns])
                        train_encoded = pd.DataFrame(ohe.transform(chunk_train_common[categorical_columns]), columns=ohe.get_feature_names_out())
                        test_encoded = pd.DataFrame(ohe.transform(chunk_test_common[categorical_columns]), columns=ohe.get_feature_names_out())
                        chunk_train_common = chunk_train_common.drop(columns=categorical_columns)
                        chunk_test_common = chunk_test_common.drop(columns=categorical_columns)
                        chunk_train_common = pd.concat([chunk_train_common, train_encoded], axis=1)
                        chunk_test_common = pd.concat([chunk_test_common, test_encoded], axis=1)

                elif encoding_method == 'ordinal':
                    categorical_columns = chunk_train_common.select_dtypes(include=['object']).columns
                    chunk_train_common[categorical_columns] = oe.fit_transform(chunk_train_common[categorical_columns])
                    chunk_test_common[categorical_columns] = oe.transform(chunk_test_common[categorical_columns])

                elif encoding_method == 'frequency':
                    for col in chunk_train_common.columns:
                        if chunk_train_common[col].dtype == 'object':
                            freq = chunk_train_common[col].value_counts() / len(chunk_train_common)
                            chunk_train_common[col] = chunk_train_common[col].map(freq)
                            chunk_test_common[col] = chunk_test_common[col].map(freq).fillna(0)

                elif encoding_method == 'target':
                    if target_columns:
                        mean_target = chunk_train.groupby(target_columns).mean()
                        for col in chunk_train_common.columns:
                            if chunk_train_common[col].dtype == 'object':
                                chunk_train_common[col] = chunk_train_common[col].map(mean_target[target_columns])
                                chunk_test_common[col] = chunk_test_common[col].map(mean_target[target_columns]).fillna(chunk_train_common[col].mean())

                elif encoding_method == 'binary':
                    chunk_train_common = be.fit_transform(chunk_train_common)
                    chunk_test_common = be.transform(chunk_test_common)

                elif encoding_method == 'hashing':
                    chunk_train_common = he.fit_transform(chunk_train_common)
                    chunk_test_common = he.transform(chunk_test_common)

                elif encoding_method == 'leaveoneout':
                    if target_columns:
                        chunk_train_common = loo.fit_transform(chunk_train_common, chunk_train[target_columns])
                        chunk_test_common = loo.transform(chunk_test_common)

                elif encoding_method == 'woe':
                    if target_columns:
                        chunk_train_common = woe.fit_transform(chunk_train_common, chunk_train[target_columns])
                        chunk_test_common = woe.transform(chunk_test_common)

                elif encoding_method == 'polynomial':
                    chunk_train_common = pe.fit_transform(chunk_train_common)
                    chunk_test_common = pe.transform(chunk_test_common)

                # Collect target labels if specified
                if target_columns:
                    for target in target_columns:
                        if target in chunk_train.columns:
                            train_labels = chunk_train[target]
                            train_labels_list[target].append(train_labels)

                # Align train and test chunks to ensure they have the same columns
                chunk_train_common, chunk_test_common = chunk_train_common.align(chunk_test_common, join='inner', axis=1)

                # Drop columns with all NaN values after alignment
                chunk_train_common.dropna(axis=1, how='all', inplace=True)
                chunk_test_common.dropna(axis=1, how='all', inplace=True)

                if not all(chunk_train_common.columns == chunk_test_common.columns):
                    continue

                if chunk_train_common.empty or chunk_test_common.empty:
                    continue

                # Combine the common columns with the target columns for the training set
                if target_columns:
                    chunk_train = pd.concat([chunk_train_common, chunk_train[target_columns]], axis=1)
                else:
                    chunk_train = chunk_train_common

                chunk_train.to_csv(output_train_file, mode='a', index=False, header=not train_header_written)
                chunk_test_common.to_csv(output_test_file, mode='a', index=False, header=not test_header_written)
                train_header_written = True
                test_header_written = True

                rows_processed += len(chunk_train_common)
                pbar.update(len(chunk_train_common))
                del chunk_train, chunk_test_common
                gc.collect()

            # If target columns are specified, append them to the output train file
            if target_columns:
                train_data = pd.read_csv(output_train_file, on_bad_lines='skip')
                for target in target_columns:
                    if target in train_labels_list:
                        target_train = pd.concat(train_labels_list[target], ignore_index=True)
                        train_data[target] = target_train
                train_data.to_csv(output_train_file, index=False)

        except Exception as e:
            print(f"An error occurred: {e}")

    # Calculate total files for progress bar
    total_files = sum([len(files) for r, d, files in os.walk(input_dir) if any('application_train.csv' in file or 'application_test.csv' in file for file in files)])
    print(f"Total files to process: {total_files}")
    pbar = tqdm(total=total_files * chunk_size, desc="Processing files")

    for root, dirs, files in os.walk(input_dir):
        print(f"Checking directory: {root}")
        if "/outliers" not in root:
            train_file = None
            test_file = None

            for file in files:
                # Identify train and test files
                if 'application_train.csv' in file:
                    train_file = file
                elif 'application_test.csv' in file:
                    test_file = file

            if not train_file or not test_file:
                continue

            input_train_file = os.path.join(root, train_file)
            input_test_file = os.path.join(root, test_file)

            for encoding_method in encoding_methods:
                imputation_method = root.split('/')[-2]
                outlier_method = root.split('/')[-1]

                # Adjust path logic based on test mode
                if test_mode:
                    output_sub_dir = os.path.join(output_dir, testing_sub_folder_name, imputation_method, outlier_method, encoding_method)
                else:
                    output_sub_dir = os.path.join(output_dir, imputation_method, outlier_method, encoding_method)

                # Ensure there are no duplicate subdirectory names
                output_sub_dir = os.path.normpath(output_sub_dir)
                os.makedirs(output_sub_dir, exist_ok=True)

                output_train_file = os.path.join(output_sub_dir, train_file)
                output_test_file = os.path.join(output_sub_dir, test_file)

                # Check if files exist before processing
                if not os.path.exists(input_train_file) or not os.path.exists(input_test_file):
                    print(f"File not found: {input_train_file} or {input_test_file}")
                    continue

                process_file(input_train_file, input_test_file, output_train_file, output_test_file, encoding_method, pbar)

    pbar.close()

### Call

In [ ]:
encode_and_align_chunked(
    input_dir=LOCAL_EXPORT_MERGED_DATAFRAMES_FOLDER_PATH,
    output_dir=LOCAL_EXPORT_ENCODED_AND_ALIGNED_FOLDER_PATH,
    target_columns=["TARGET"],
    encoding_methods=["onehot", "label", "ordinal"],  # spécifiez les types d'encodage souhaités
    chunk_size=GENERAL_CHUNK_SIZE,
    test_mode=TESTING_MODE,
    max_rows=TESTING_MODE_MAX_LINES,
    testing_sub_folder_name=TESTING_MODE_SUB_FOLDER_NAME
)

In [ ]:
test_encoding_prev = pd.read_csv("/content/exports/encoded_and_aligned/testing_data/Isolation Forest/ffill/onehot/application_test_merged.csv")
test_encoding_prev_label = pd.read_csv("/content/exports/encoded_and_aligned/testing_data/Isolation Forest/ffill/label/application_test_merged.csv")
test_encoding_prev_ordinal = pd.read_csv("/content/exports/encoded_and_aligned/testing_data/Isolation Forest/ffill/ordinal/application_test_merged.csv")


In [ ]:
test_encoding_prev.head()

In [ ]:
test_encoding_prev_label.head()

In [ ]:
test_encoding_prev_ordinal.head()

## Back to Exploratory Data Analysis

### Anomalies

One problem we always want to be on the lookout for when doing EDA is anomalies within the data. These may be due to mis-typed numbers, errors in measuring equipment, or they could be valid but extreme measurements. One way to support anomalies quantitatively is by looking at the statistics of a column using the `describe` method. The numbers in the `DAYS_BIRTH` column are negative because they are recorded relative to the current loan application. To see these stats in years, we can mutliple by -1 and divide by the number of days in a year:



In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()

Those ages look reasonable. There are no outliers for the age on either the high or low end. How about the days of employment?

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

That doesn't look right! The maximum value (besides being positive) is about 1000 years!

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

Just out of curiousity, let's subset the anomalous clients and see if they tend to have higher or low rates of default than the rest of the clients.

## Manual check patch after encoding

### Method

In [ ]:
def handle_anomalies(df, column, anomaly_value):
    # Create an anomalous flag column
    # df[f'{column}_ANOM'] = df[column] == anomaly_value
    print(f"Anomalies found in column {column}: {(df[f'{column}'] == anomaly_value).sum()}")
    # Replace the anomalous values with np.nan
    df[column].replace({anomaly_value: 0}, inplace=True)
    print(f"Anomalies found in column after patch {column}: {(df[f'{column}'] == anomaly_value).sum()}")

    return df

def process_files_in_directory(base_path, anomaly_column, anomaly_value, output_folder, max_rows=None, testing=False, testing_sub_path_name="testing_data"):
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                # Read the CSV file
                df = pd.read_csv(file_path, nrows=max_rows)
                # Handle anomalies
                df = handle_anomalies(df, anomaly_column, anomaly_value)

                # Determine the output path
                relative_path = os.path.relpath(root, base_path)
                imputation_method = root.split('/')[-3]
                outlier_method = root.split('/')[-2]
                encoding_method = root.split('/')[-1]

                if testing:
                    output_dir = os.path.join(output_folder, testing_sub_path_name, imputation_method, outlier_method, encoding_method)
                else:
                    output_dir = os.path.join(output_folder, imputation_method, outlier_method, encoding_method)

                os.makedirs(output_dir, exist_ok=True)
                output_file_path = os.path.join(output_dir, file)

                # Save the processed DataFrame
                df.to_csv(output_file_path, index=False)
                print(f"Saved processed file to: {output_file_path}")

### Call

In [ ]:
process_files_in_directory(
        base_path=LOCAL_EXPORT_ENCODED_AND_ALIGNED_FOLDER_PATH,
        anomaly_column="DAYS_EMPLOYED",
        anomaly_value=365243,
        output_folder=LOCAL_EXPORT_MANUAL_CHECK_PATCH_FOLDER_PATH,
        max_rows=TESTING_MODE_MAX_LINES,  # Set to None to process all rows
        testing=TESTING_MODE,
        testing_sub_path_name=TESTING_MODE_SUB_FOLDER_NAME)

Well that is extremely interesting! It turns out that the anomalies have a lower rate of default.

Handling the anomalies depends on the exact situation, with no set rules. One of the safest approaches is just to set the anomalies to a missing value and then have them filled in (using Imputation) before machine learning. In this case, since all the anomalies have the exact same value, we want to fill them in with the same value in case all of these loans share something in common. The anomalous values seem to have some importance, so we want to tell the machine learning model if we did in fact fill in these values. As a solution, we will fill in the anomalous values with not a number (`np.nan`) and then create a new boolean column indicating whether or not the value was anomalous.



In [ ]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

The distribution looks to be much more in line with what we would expect, and we also have created a new column to tell the model that these values were originally anomalous (becuase we will have to fill in the nans with some value, probably the median of the column). The other columns with `DAYS` in the dataframe look to be about what we expect with no obvious outliers.

As an extremely important note, anything we do to the training data we also have to do to the testing data. Let's make sure to create the new column and fill in the existing column with `np.nan` in the testing data.

In [ ]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

### Correlations

Now that we have dealt with the categorical variables and the outliers, let's continue with the EDA. One way to try and understand the data is by looking for correlations between the features and the target. We can calculate the Pearson correlation coefficient between every variable and the target using the `.corr` dataframe method.

The correlation coefficient is not the greatest method to represent "relevance" of a feature, but it does give us an idea of possible relationships within the data. Some [general interpretations of the absolute value of the correlation coefficent](http://www.statstutor.ac.uk/resources/uploaded/pearsons.pdf) are:


* .00-.19 “very weak”
*  .20-.39 “weak”
*  .40-.59 “moderate”
*  .60-.79 “strong”
* .80-1.0 “very strong”


In [ ]:
# Find correlations with the target and sort
correlations = app_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [ ]:
correlations.to_csv("correlations_extrac.csv", index=True)

Let's take a look at some of more significant correlations: the `DAYS_BIRTH` is the most positive correlation. (except for `TARGET` because the correlation of a variable with itself is always 1!) Looking at the documentation, `DAYS_BIRTH` is the age in days of the client at the time of the loan in negative days (for whatever reason!). The correlation is positive, but the value of this feature is actually negative, meaning that as the client gets older, they are less likely to default on their loan (ie the target == 0). That's a little confusing, so we will take the absolute value of the feature and then the correlation will be negative.

### Effect of Age on Repayment

In [ ]:
# Find the correlation of the positive days since birth and target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

As the client gets older, there is a negative linear relationship with the target meaning that as clients get older, they tend to repay their loans on time more often.

Let's start looking at this variable. First, we can make a histogram of the age. We will put the x axis in years to make the plot a little more understandable.

In [ ]:
# Set the style of plots
plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

By itself, the distribution of age does not tell us much other than that there are no outliers as all the ages are reasonable. To visualize the effect of the age on the target, we will next make a [kernel density estimation plot](https://en.wikipedia.org/wiki/Kernel_density_estimation) (KDE) colored by the value of the target. A [kernel density estimate plot shows the distribution of a single variable](https://chemicalstatistician.wordpress.com/2013/06/09/exploratory-data-analysis-kernel-density-estimation-in-r-on-ozone-pollution-data-in-new-york-and-ozonopolis/) and can be thought of as a smoothed histogram (it is created by computing a kernel, usually a Gaussian, at each data point and then averaging all the individual kernels to develop a single smooth curve). We will use the seaborn `kdeplot` for this graph.

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

The target == 1 curve skews towards the younger end of the range. Although this is not a significant correlation (-0.07 correlation coefficient), this variable is likely going to be useful in a machine learning model because it does affect the target. Let's look at this relationship in another way: average failure to repay loans by age bracket.

To make this graph, first we `cut` the age category into bins of 5 years each. Then, for each bin, we calculate the average value of the target, which tells us the ratio of loans that were not repaid in each age category.

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

There is a clear trend: younger applicants are more likely to not repay the loan! The rate of failure to repay is above 10% for the youngest three age groups and beolow 5% for the oldest age group.

This is information that could be directly used by the bank: because younger clients are less likely to repay the loan, maybe they should be provided with more guidance or financial planning tips. This does not mean the bank should discriminate against younger clients, but it would be smart to take precautionary measures to help younger clients pay on time.

### Exterior Sources

The 3 variables with the strongest negative correlations with the target are `EXT_SOURCE_1`, `EXT_SOURCE_2`, and `EXT_SOURCE_3`.
According to the documentation, these features represent a "normalized score from external data source". I'm not sure what this exactly means, but it may be a cumulative sort of credit rating made using numerous sources of data.

Let's take a look at these variables.

First, we can show the correlations of the `EXT_SOURCE` features with the target and with each other.

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize = (8, 6))

# Heatmap of correlations
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

All three `EXT_SOURCE` featureshave negative correlations with the target, indicating that as the value of the `EXT_SOURCE` increases, the client is more likely to repay the loan. We can also see that `DAYS_BIRTH` is positively correlated with `EXT_SOURCE_1` indicating that maybe one of the factors in this score is the client age.

Next we can look at the distribution of each of these features colored by the value of the target. This will let us visualize the effect of this variable on the target.

In [ ]:
plt.figure(figsize = (10, 12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):

    # create a new subplot for each source
    plt.subplot(3, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')

    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');

plt.tight_layout(h_pad = 2.5)


`EXT_SOURCE_3` displays the greatest difference between the values of the target. We can clearly see that this feature has some relationship to the likelihood of an applicant to repay a loan. The relationship is not very strong (in fact they are all [considered very weak](http://www.statstutor.ac.uk/resources/uploaded/pearsons.pdf), but these variables will still be useful for a machine learning model to predict whether or not an applicant will repay a loan on time.

## Pairs Plot

As a final exploratory plot, we can make a pairs plot of the `EXT_SOURCE` variables and the `DAYS_BIRTH` variable. The [Pairs Plot](https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166) is a great exploration tool because it lets us see relationships between multiple pairs of variables as well as distributions of single variables. Here we are using the seaborn visualization library and the PairGrid function to create a Pairs Plot with scatterplots on the upper triangle, histograms on the diagonal, and 2D kernel density plots and correlation coefficients on the lower triangle.

If you don't understand this code, that's all right! Plotting in Python can be overly complex, and for anything beyond the simplest graphs, I usually find an existing implementation and adapt the code (don't repeat yourself)!

In [ ]:
# Copy the data for plotting
plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# Add in the age of the client in years
plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().loc[:100000, :]

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'TARGET',
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05);

In this plot, the red indicates loans that were not repaid and the blue are loans that are paid. We can see the different relationships within the data. There does appear to be a moderate positive linear relationship between the `EXT_SOURCE_1` and the `DAYS_BIRTH` (or equivalently `YEARS_BIRTH`), indicating that this feature may take into account the age of the client.

In [ ]:
columns_to_keep = [
    'DAYS_BIRTH',
    'DAYS_EMPLOYED',
    'NAME_FAMILY_STATUS',
    'CODE_GENDER',
    'CNT_CHILDREN',
    'CNT_FAM_MEMBERS',
    'NAME_INCOME_TYPE',
    'OCCUPATION_TYPE',
    'DAYS_CREDIT',
    'DAYS_CREDIT_ENDDATE',
    'DAYS_CREDIT_UPDATE',
    'DAYS_DECISION',
    'AMT_CREDIT',
    'AMT_ANNUITY',
    'AMT_GOODS_PRICE',
    'DAYS_LAST_DUE',
    'DAYS_LAST_DUE_1ST_VERSION',
    'DAYS_TERMINATION',
    'DAYS_ID_PUBLISH',
    'DAYS_INSTALMENT',
    'DAYS_LAST_PHONE_CHANGE',
    'CNT_INSTALMENT',
    'CNT_INSTALMENT_FUTURE',
    'CNT_INSTALMENT_MATURE_CUM',
    'CNT_PAYMENT',
    'SK_DPD',
    'SK_DPD_DEF',
    'CREDIT_DAY_OVERDUE',
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT_SUM',
    'AMT_CREDIT_SUM_DEBT',
    'AMT_CREDIT_SUM_LIMIT',
    'AMT_CREDIT_SUM_OVERDUE',
    'AMT_REQ_CREDIT_BUREAU_YEAR',
    'REGION_RATING_CLIENT',
    'REGION_RATING_CLIENT_W_CITY',
    'REG_CITY_NOT_WORK_CITY',
    'REG_CITY_NOT_LIVE_CITY',
    'REG_REGION_NOT_WORK_REGION',
    'REG_REGION_NOT_LIVE_REGION',
    'LIVE_CITY_NOT_WORK_CITY',
    'LIVE_REGION_NOT_WORK_REGION',
    'FLAG_OWN_REALTY',
    'OWN_CAR_AGE',
    'FLAG_DOCUMENT_3',
    'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE',
    'AGE_EMPLOYED_RATIO',
    'CREDIT_INCOME_RATIO',
    'ANNUITY_INCOME_RATIO',
    'CREDIT_ANNUITY_RATIO',
    'CHILDREN_RATIO',
    'INCOME_PER_FAMILY_MEMBER'
]

# Entraîner un modèle de forêt aléatoire pour obtenir l'importance des features

In [ ]:
model = RandomForestClassifier()

In [ ]:
dfs_test = pd.read_csv(f"{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged.csv", chunksize=1000)

In [ ]:
for i, chunk in enumerate(dfs_test):
    if i == 1001:
        break
    model.fit(chunk.drop(columns=['TARGET']), chunk['TARGET'])

In [ ]:
chunksize = 10000  # Définir la taille du lot
for i, chunk in enumerate(pd.read_csv(f"{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged.csv", chunksize=chunksize)):
    # Traitement du lot
    # processed_chunk = process(chunk)
    model.fit(app_train.drop(columns=['TARGET']), app_train['TARGET'])

    # Exporter le lot traité en CSV
    # processed_chunk.to_csv(f'${LOCAL_EXPORT_FOLDER_PATH}/processed_chunk_{i}.csv', index=False)

    # Libérer la mémoire
    del chunk
    gc.collect()

In [ ]:
function training(app_train):
    model.fit(app_train.drop(columns=['TARGET']), app_train['TARGET'])

# Importance des features

In [ ]:
feature_importances = pd.DataFrame(model.feature_importances_, index=app_train.drop(columns=['TARGET']).columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances.head(20))


# Entraîner le modèle avec les nouvelles features

In [ ]:
features = app_train.drop(columns=['TARGET'])
target = app_train['TARGET']
model = RandomForestClassifier()
scores = cross_val_score(model, features, target, cv=5, scoring='roc_auc')

print(f"Mean ROC AUC Score: {scores.mean()}")


Étape 4: Validation et Évaluation
Valider et évaluer l'impact des nouvelles features sur la performance du modèle.

4.1 Validation Croisée
Utiliser la validation croisée pour évaluer la performance du modèle.

python
Copy code


features = app_train.drop(columns=['TARGET'])
target = app_train['TARGET']
model = RandomForestClassifier()
scores = cross_val_score(model, features, target, cv=5, scoring='roc_auc')

print(f"Mean ROC AUC Score: {scores.mean()}")
4.2 Évaluation de la Performance
Comparer les performances du modèle avec et sans les nouvelles features.

python
Copy code
# Modèle avec les nouvelles features
model_with_features = RandomForestClassifier()
model_with_features.fit(features, target)
scores_with_features = cross_val_score(model_with_features, features, target, cv=5, scoring='roc_auc')

# Modèle sans les nouvelles features
selected_features = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'AMT_CREDIT', 'AMT_ANNUITY', 'SK_DPD', 'CREDIT_DAY_OVERDUE', 'AMT_INCOME_TOTAL']
features_without = app_train[selected_features]
model_without_features = RandomForestClassifier()
model_without_features.fit(features_without, target)
scores_without_features = cross_val_score(model_without_features, features_without, target, cv=5, scoring='roc_auc')

print(f"Mean ROC AUC Score with features: {scores_with_features.mean()}")
print(f"Mean ROC AUC Score without features: {scores_without_features.mean()}")

## Polynomial Features

One simple feature construction method is called [polynomial features](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). In this method, we make features that are powers of existing features as well as interaction terms between existing features. For example, we can create variables `EXT_SOURCE_1^2` and `EXT_SOURCE_2^2` and also variables such as `EXT_SOURCE_1` x `EXT_SOURCE_2`, `EXT_SOURCE_1` x `EXT_SOURCE_2^2`, `EXT_SOURCE_1^2` x   `EXT_SOURCE_2^2`, and so on. These features that are a combination of multiple individual variables are called [interaction terms](https://en.wikipedia.org/wiki/Interaction_(statistics) because they  capture the interactions between variables. In other words, while two variables by themselves  may not have a strong influence on the target, combining them together into a single interaction variable might show a relationship with the target. [Interaction terms are commonly used in statistical models](https://www.theanalysisfactor.com/interpreting-interactions-in-regression/) to capture the effects of multiple variables, but I do not see them used as often in machine learning. Nonetheless, we can try out a few to see if they might help our model to predict whether or not a client will repay a loan.

Jake VanderPlas writes about [polynomial features in his excellent book Python for Data Science](https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html) for those who want more information.

In the following code, we create polynomial features using the `EXT_SOURCE` variables and the `DAYS_BIRTH` variable. [Scikit-Learn has a useful class called `PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) that creates the polynomials and the interaction terms up to a specified degree. We can use a degree of 3 to see the results (when we are creating polynomial features, we want to avoid using too high of a degree, both because the number of features scales exponentially with the degree, and because we can run into [problems with overfitting](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html#sphx-glr-auto-examples-model-selection-plot-underfitting-overfitting-py)).

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from tqdm import tqdm

def generate_polynomial_features(input_file_train, input_file_test, output_file_train, output_file_test, features, degree=3, chunk_size=10000, target_column=None, id_column=None, add_features=True):
    train_header_written = False
    test_header_written = False

    # Imputer pour gérer les valeurs manquantes
    imputer = SimpleImputer(strategy='median')

    # Calculer le nombre total de lignes pour la barre de progression
    total_lines_train = sum(1 for _ in open(input_file_train)) - 1  # -1 pour l'en-tête
    total_lines_test = sum(1 for _ in open(input_file_test)) - 1  # -1 pour l'en-tête

    # Lire les fichiers en chunks
    chunk_train_iterator = pd.read_csv(input_file_train, chunksize=chunk_size)
    chunk_test_iterator = pd.read_csv(input_file_test, chunksize=chunk_size)

    with tqdm(total=total_lines_train, desc="Traitement des chunks") as pbar:
        for chunk_train, chunk_test in zip(chunk_train_iterator, chunk_test_iterator):
            # Vérifier la présence des colonnes de features
            missing_features_train = [feature for feature in features if feature not in chunk_train.columns]
            missing_features_test = [feature for feature in features if feature not in chunk_test.columns]

            if missing_features_train:
                print(f"Colonnes manquantes dans le chunk d'entraînement : {missing_features_train}")
                print(f"Colonnes disponibles dans le chunk d'entraînement : {chunk_train.columns.to_list()}")
                pbar.update(len(chunk_train))
                continue

            if missing_features_test:
                print(f"Colonnes manquantes dans le chunk de test : {missing_features_test}")
                print(f"Colonnes disponibles dans le chunk de test : {chunk_test.columns.to_list()}")
                pbar.update(len(chunk_train))
                continue

            # Séparer les features et la colonne cible si elle est spécifiée
            target_train = None
            if target_column and target_column in chunk_train.columns:
                target_train = chunk_train[target_column]
                chunk_train = chunk_train.drop(columns=[target_column])

            # Séparer la colonne ID si elle est spécifiée
            id_train = id_test = None
            if id_column and id_column in chunk_train.columns:
                id_train = chunk_train[id_column]
                id_test = chunk_test[id_column]
                chunk_train = chunk_train.drop(columns=[id_column])
                chunk_test = chunk_test.drop(columns=[id_column])

            # Sélectionner les features à transformer
            chunk_train_features = chunk_train[features]
            chunk_test_features = chunk_test[features]

            # Vérifier que les DataFrames de features ne sont pas vides
            if chunk_train_features.shape[1] == 0 or chunk_test_features.shape[1] == 0:
                print("Les DataFrames de features sont vides après la sélection des colonnes spécifiées. Chunk ignoré.")
                pbar.update(len(chunk_train))
                continue

            # Imputer les valeurs manquantes
            chunk_train_features = imputer.fit_transform(chunk_train_features)
            chunk_test_features = imputer.transform(chunk_test_features)

            # Créer l'objet PolynomialFeatures avec le degré spécifié
            poly_transformer = PolynomialFeatures(degree=degree)

            try:
                # Entraîner et transformer les features
                poly_train_features = poly_transformer.fit_transform(chunk_train_features)
                poly_test_features = poly_transformer.transform(chunk_test_features)
            except ValueError as e:
                print(f"Erreur lors de la transformation polynomiale : {e}")
                print("Chunk ignoré.")
                pbar.update(len(chunk_train))
                continue

            # Créer les DataFrames avec les nouvelles features polynomiales
            poly_feature_names = poly_transformer.get_feature_names_out(features)
            poly_train_features = pd.DataFrame(poly_train_features, columns=poly_feature_names)
            poly_test_features = pd.DataFrame(poly_test_features, columns=poly_feature_names)

            if add_features:
                # Ajouter les nouvelles features polynomiales aux DataFrames originaux
                chunk_train = pd.concat([chunk_train, poly_train_features], axis=1)
                chunk_test = pd.concat([chunk_test, poly_test_features], axis=1)
            else:
                # Remplacer les features existantes par les nouvelles features polynomiales
                chunk_train = poly_train_features
                chunk_test = poly_test_features

            # Réinsérer les colonnes ID si elles étaient spécifiées
            if id_column and id_train is not None:
                chunk_train[id_column] = id_train
                chunk_test[id_column] = id_test

            # Réinsérer la colonne cible si elle était spécifiée
            if target_column and target_train is not None:
                chunk_train[target_column] = target_train

            # Exporter les chunks avec les features polynomiales
            chunk_train.to_csv(output_file_train, mode='a', index=False, header=not train_header_written)
            chunk_test.to_csv(output_file_test, mode='a', index=False, header=not test_header_written)
            train_header_written = True
            test_header_written = True

            # Nettoyage de la mémoire
            del poly_train_features, poly_test_features, chunk_train, chunk_test
            gc.collect()

            # Mettre à jour la barre de progression
            pbar.update(chunk_size)



In [ ]:
features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']

In [ ]:
input_file_tain_to_poly = f'{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged_encoded.csv'
input_file_test_to_poly = f'{LOCAL_EXPORT_FOLDER_PATH}/application_test_merged_encoded.csv'
output_file_train_poly = f'{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged_with_new_features_encoded_poly_add.csv'
output_file_test_poly = f'{LOCAL_EXPORT_FOLDER_PATH}/application_test_merged_with_new_features_encoded_poly_add.csv'

In [ ]:
target_column = 'TARGET'
id_column = 'SK_ID_CURR'
generate_polynomial_features(input_file_tain_to_poly, input_file_test_to_poly, output_file_train_poly, output_file_test_poly, features, degree=3, target_column=target_column, id_column=id_column, add_features=True)

### Verify of correlations

In [ ]:
# Calcul des corrélations avec plusieurs colonnes cibles
def calculate_correlations_with_targets(df, target_columns):
    correlations = {}
    for target in target_columns:
        correlations[target] = df.corr()[target].abs().sort_values(ascending=False)
    return correlations

# Sélection des top features par corrélation avec plusieurs colonnes cibles
def select_top_features_by_correlation(df, target_columns, top_n):
    all_corrs = calculate_correlations_with_targets(df, target_columns)
    top_features = set()
    for target, corrs in all_corrs.items():
        top_features.update(corrs.index[1:top_n+1])  # Exclure la colonne cible
    return df[list(top_features)]

# Exemple d'utilisation pour top 100 features par corrélation
target_columns = ['TARGET']
df_train_top100_corr = select_top_features_by_correlation(df_train_poly, target_columns, 100)
df_test_top100_corr = df_test_poly[df_train_top100_corr.columns]


## Best features

In [ ]:
# Exporter en CSV
export_to_csv_in_chunks(df_train_top100_corr, f'{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged_encoded_with_features_poly_add_top100_corr.csv')
export_to_csv_in_chunks(df_test_top100_corr, f'{LOCAL_EXPORT_FOLDER_PATH}/application_test_merged_encoded_with_features_poly_add_top100_corr.csv')


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.feature_selection import RFE, mutual_info_classif, chi2, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import joblib  # To save the trained model

def calculate_total_rows(file_path):
    return sum(1 for _ in open(file_path)) - 1  # subtract 1 for header

def impute_missing_values(chunk):
    imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
    try:
        imputed_chunk = pd.DataFrame(imputer.fit_transform(chunk), columns=chunk.columns)
    except ValueError as e:
        print(f"Imputation error: {e}")
        return None
    return imputed_chunk

def calculate_feature_importance_via_chunks(input_csv, target_column, method, chunk_size=10000):
    aggregated_importances = None
    total_chunks = 0
    total_rows = calculate_total_rows(input_csv)
    processed_rows = 0

    for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
        print(f"Columns in chunk before imputation: {chunk.columns}")  # Debug: print columns
        chunk = impute_missing_values(chunk)

        if chunk is None or chunk.shape[1] != len(chunk.columns):
            print(f"Skipping problematic chunk with shape {chunk.shape if chunk is not None else 'None'}")
            continue

        print(f"Columns in chunk after imputation: {chunk.columns}")  # Debug: print columns

        if target_column not in chunk.columns:
            print(f"Target column '{target_column}' not found in the columns: {chunk.columns}")
            continue

        if chunk[target_column].nunique() < 2:
            print(f"Skipping chunk with less than 2 classes in target column: {chunk[target_column].unique()}")
            continue

        X_train = chunk.drop(columns=[target_column])
        y_train = chunk[target_column]

        if method == 'random_forest':
            model = RandomForestClassifier()
            model.fit(X_train, y_train)
            feature_importances = pd.Series(model.feature_importances_, index=X_train.columns)
        elif method == 'rfe':
            model = LogisticRegression(max_iter=1000)
            rfe = RFE(model, n_features_to_select=20)
            rfe.fit(X_train, y_train)
            feature_importances = pd.Series(rfe.ranking_, index=X_train.columns).sort_values()
        elif method == 'lasso':
            model = Lasso(alpha=0.01)
            model.fit(X_train, y_train)
            feature_importances = pd.Series(model.coef_, index=X_train.columns).abs()
        elif method == 'mutual_info':
            mutual_info = mutual_info_classif(X_train, y_train)
            feature_importances = pd.Series(mutual_info, index=X_train.columns)
        elif method == 'chi2':
            chi_scores, _ = chi2(X_train, y_train)
            feature_importances = pd.Series(chi_scores, index=X_train.columns)
        elif method == 'f_classif':
            f_values, _ = f_classif(X_train, y_train)
            feature_importances = pd.Series(f_values, index=X_train.columns)
        elif method == 'select_from_model':
            selector = SelectFromModel(RandomForestClassifier(), max_features=100)
            selector.fit(X_train, y_train)
            feature_importances = pd.Series(selector.estimator_.feature_importances_, index=X_train.columns)

        if aggregated_importances is None:
            aggregated_importances = feature_importances
        else:
            aggregated_importances = aggregated_importances.add(feature_importances, fill_value=0)

        total_chunks += 1
        processed_rows += chunk_size
        progress = (processed_rows / total_rows) * 100
        print(f"Progress: {min(progress, 100):.2f}%")

    aggregated_importances /= total_chunks
    sorted_importances = aggregated_importances.sort_values(ascending=False)
    top_features = sorted_importances.head(100)

    return top_features

def save_top_features(top_features, output_csv):
    top_features.to_csv(output_csv, header=True)

def save_filtered_dataset(input_csv, top_features, output_csv, chunk_size=10000):
    top_features = top_features.index
    total_rows = calculate_total_rows(input_csv)
    processed_rows = 0

    first_chunk = True
    for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
        chunk = impute_missing_values(chunk)

        if chunk is None or chunk.shape[1] != len(chunk.columns):
            print(f"Skipping problematic chunk with shape {chunk.shape if chunk is not None else 'None'}")
            continue

        print(f"Columns in chunk: {chunk.columns}")  # Debug: print columns
        filtered_chunk = chunk[top_features]
        filtered_chunk.to_csv(output_csv, mode='a', header=first_chunk, index=False)
        first_chunk = False

        processed_rows += chunk_size
        progress = (processed_rows / total_rows) * 100
        print(f"Progress: {min(progress, 100):.2f}%")

def retrain_model_with_top_features(train_csv, test_csv, top_features_csv, target_column, model_output, chunk_size=10000):
    top_features = pd.read_csv(top_features_csv, index_col=0).index

    X_train = pd.DataFrame()
    y_train = pd.Series(dtype='int')

    total_train_rows = calculate_total_rows(train_csv)
    processed_train_rows = 0

    for chunk in pd.read_csv(train_csv, chunksize=chunk_size):
        chunk = impute_missing_values(chunk)

        if chunk is None or chunk.shape[1] != len(chunk.columns):
            print(f"Skipping problematic chunk with shape {chunk.shape if chunk is not None else 'None'}")
            continue

        print(f"Columns in chunk: {chunk.columns}")  # Debug: print columns
        if target_column not in chunk.columns:
            print(f"Target column '{target_column}' not found in the columns: {chunk.columns}")
            continue

        if chunk[target_column].nunique() < 2:
            print(f"Skipping chunk with less than 2 classes in target column: {chunk[target_column].unique()}")
            continue

        X_chunk = chunk[top_features]
        y_chunk = chunk[target_column]

        X_train = pd.concat([X_train, X_chunk], ignore_index=True)
        y_train = pd.concat([y_train, y_chunk], ignore_index=True)

        processed_train_rows += chunk_size
        progress = (processed_train_rows / total_train_rows) * 100
        print(f"Training Progress: {min(progress, 100):.2f}%")

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    joblib.dump(model, model_output)
    print(f'Model saved to {model_output}')

    X_test = pd.DataFrame()
    y_test = pd.Series(dtype='int')

    total_test_rows = calculate_total_rows(test_csv)
    processed_test_rows = 0

    for chunk in pd.read_csv(test_csv, chunksize=chunk_size):
        chunk = impute_missing_values(chunk)

        if chunk is None or chunk.shape[1] != len(chunk.columns):
            print(f"Skipping problematic chunk with shape {chunk.shape if chunk is not None else 'None'}")
            continue

        print(f"Columns in chunk: {chunk.columns}")  # Debug: print columns
        if target_column not in chunk.columns:
            print(f"Target column '{target_column}' not found in the columns: {chunk.columns}")
            continue

        if chunk[target_column].nunique() < 2:
            print(f"Skipping chunk with less than 2 classes in target column: {chunk[target_column].unique()}")
            continue

        X_chunk = chunk[top_features]
        y_chunk = chunk[target_column]

        X_test = pd.concat([X_test, X_chunk], ignore_index=True)
        y_test = pd.concat([y_test, y_chunk], ignore_index=True)

        processed_test_rows += chunk_size
        progress = (processed_test_rows / total_test_rows) * 100
        print(f"Testing Progress: {min(progress, 100):.2f}%")

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with top features: {accuracy}")

    return model

def run_feature_selection_and_retraining(train_csv, test_csv, target_column, methods, chunk_size=10000):
    for method in methods:
        top_features = calculate_feature_importance_via_chunks(train_csv, target_column, method, chunk_size)
        output_csv = f'top_features_{method}.csv'
        save_top_features(top_features, output_csv)
        print(f'Top features saved to {output_csv} for method {method}')

        filtered_train_csv = f'filtered_train_{method}.csv'
        save_filtered_dataset(train_csv, top_features, filtered_train_csv, chunk_size)
        print(f'Filtered training data saved to {filtered_train_csv} for method {method}')

        filtered_test_csv = f'filtered_test_{method}.csv'
        save_filtered_dataset(test_csv, top_features, filtered_test_csv, chunk_size)
        print(f'Filtered test data saved to {filtered_test_csv} for method {method}')

        model_output = f'model_{method}.joblib'
        retrain_model_with_top_features(train_csv, test_csv, output_csv, target_column, model_output, chunk_size)

In [ ]:
test12312  = pd.read_csv(f'{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged_with_new_features_encoded_poly_add.csv')

In [ ]:
test12312.columns.to_list()

In [ ]:
feature_selection_and_training_input_train_csv = f'{LOCAL_EXPORT_FOLDER_PATH}/application_train_merged_with_new_features_encoded_poly_add.csv'
feature_selection_and_training_output_test_csv = f'{LOCAL_EXPORT_FOLDER_PATH}/application_test_merged_with_new_features_encoded_poly_add.csv'
target_column = '1'
methods = ['random_forest', 'rfe', 'lasso', 'mutual_info', 'chi2', 'f_classif', 'select_from_model']

run_feature_selection_and_retraining(feature_selection_and_training_input_train_csv, feature_selection_and_training_output_test_csv, target_column, methods, chunk_size=10000)

In [ ]:
top_features_random_forest = pd.read_csv(f'/content/top_features_random_forest.csv')

#### Random Forest:

Description : Utilise l'importance des features déterminée par un modèle de forêt aléatoire pour sélectionner les features les plus importantes.
Formule : Importance d'une feature = Réduction moyenne de l'impureté (Gini ou Entropie) sur tous les arbres.

#### Recursive Feature Elimination (RFE):

Description : Sélectionne les features en itérant sur le modèle, éliminant les features les moins importantes à chaque étape.
Formule : Basé sur le score du modèle après l'élimination de chaque feature, en boucle.

#### Lasso (Least Absolute Shrinkage and Selection Operator):

Description : Utilise la régularisation L1 qui force certains coefficients de features à devenir exactement zéro, effectuant ainsi une sélection de features.
Formule :
min
⁡
{
1
2
𝑛
∑
𝑖
=
1
𝑛
(
𝑦
𝑖
−
𝑋
𝑖
𝛽
)
2
+
𝜆
∑
𝑗
=
1
𝑝
∣
𝛽
𝑗
∣
}
min{
2n
1
​
 ∑
i=1
n
​
 (y
i
​
 −X
i
​
 β)
2
 +λ∑
j=1
p
​
 ∣β
j
​
 ∣}

#### Mutual Information:

Description : Mesure la dépendance mutuelle entre chaque feature et la variable cible.
Formule :
𝐼
(
𝑋
;
𝑌
)
=
∑
𝑥
∈
𝑋
∑
𝑦
∈
𝑌
𝑝
(
𝑥
,
𝑦
)
log
⁡
𝑝
(
𝑥
,
𝑦
)
𝑝
(
𝑥
)
𝑝
(
𝑦
)
I(X;Y)=∑
x∈X
​
 ∑
y∈Y
​
 p(x,y)log
p(x)p(y)
p(x,y)
​

#### Chi-squared (Chi2):

Description : Test statistique pour mesurer l'indépendance entre deux variables catégorielles.
Formule :
𝜒
2
=
∑
(
𝑂
𝑖
−
𝐸
𝑖
)
2
𝐸
𝑖
χ
2
 =∑
E
i
​

(O
i
​
 −E
i
​
 )
2

​


#### ANOVA F-test (f_classif):

Description : Utilise l'ANOVA pour comparer les variances entre les groupes et au sein des groupes.
Formule :
𝐹
=
variance entre les groupes
variance au sein du groupe
F=
variance au sein du groupe
variance entre les groupes
​

#### SelectFromModel:

Description : Utilise l'importance des features déterminée par un modèle (comme Lasso, Random Forest, etc.) pour sélectionner les features.
Formule : Basé sur les coefficients ou importances des features déterminés par le modèle sélectionné.

In [ ]:
top_features_random_forest.head()

In [ ]:
# Make a new dataframe for polynomial features
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# imputer for handling missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures

# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

This creates a considerable number of new features. To get the names we have to use the polynomial features `get_feature_names` method.

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

There are 35 features with individual features raised to powers up to degree 3 and interaction terms. Now, we can see whether any of these new features are correlated with the target.

In [ ]:
# Create a dataframe of the features
poly_features = pd.DataFrame(poly_features,
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2',
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

Several of the new variables have a greater (in terms of absolute magnitude) correlation with the target than the original features. When we build machine learning models, we can try with and without these features to determine if they actually help the model learn.

We will add these features to a copy of the training and testing data and then evaluate models with and without the features. Many times in machine learning, the only way to know if an approach will work is to try it out!

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test,
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2',
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

## Domain Knowledge Features

Maybe it's not entirely correct to call this "domain knowledge" because I'm not a credit expert, but perhaps we could call this "attempts at applying limited financial knowledge". In this frame of mind, we can make a couple features that attempt to capture what we think may be important for telling whether a client will default on a loan. Here I'm going to use five features that were inspired by [this script](https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features) by Aguiar:

* `CREDIT_INCOME_PERCENT`: the percentage of the credit amount relative to a client's income
* `ANNUITY_INCOME_PERCENT`: the percentage of the loan annuity relative to a client's income
* `CREDIT_TERM`:  the length of the payment in months (since the annuity is the monthly amount due
* `DAYS_EMPLOYED_PERCENT`: the percentage of the days employed relative to the client's age

Again, thanks to Aguiar and [his great script](https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features) for exploring these features.



In [ ]:
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

#### Visualize New Variables

We should explore these __domain knowledge__ variables visually in a graph. For all of these, we will make the same KDE plot colored by the value of the `TARGET`.

In [ ]:
plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):

    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')

    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');

plt.tight_layout(h_pad = 2.5)

It's hard to say ahead of time if these new features will be useful. The only way to tell for sure is to try them out!

In [ ]:
xgboost
lgbm


# Baseline

For a naive baseline, we could guess the same value for all examples on the testing set.  We are asked to predict the probability of not repaying the loan, so if we are entirely unsure, we would guess 0.5 for all observations on the test set. This  will get us a Reciever Operating Characteristic Area Under the Curve (AUC ROC) of 0.5 in the competition ([random guessing on a classification task will score a 0.5](https://stats.stackexchange.com/questions/266387/can-auc-roc-be-between-0-0-5)).

Since we already know what score we are going to get, we don't really need to make a naive baseline guess. Let's use a slightly more sophisticated model for our actual baseline: Logistic Regression.

## Logistic Regression Implementation

Here I will focus on implementing the model rather than explaining the details, but for those who want to learn more about the theory of machine learning algorithms, I recommend both [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/) and [Hands-On Machine Learning with Scikit-Learn and TensorFlow](http://shop.oreilly.com/product/0636920052289.do). Both of these books present the theory and also the code needed to make the models (in R and Python respectively). They both teach with the mindset that the best way to learn is by doing, and they are very effective!

To get a baseline, we will use all of the features after encoding the categorical variables. We will preprocess the data by filling in the missing values (imputation) and normalizing the range of the features (feature scaling). The following code performs both of these preprocessing steps.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()

# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

We will use [`LogisticRegression`from Scikit-Learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for our first model. The only change we will make from the default model settings is to lower the [regularization parameter](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), C, which controls the amount of overfitting (a lower value should decrease overfitting). This will get us slightly better results than the default `LogisticRegression`, but it still will set a low bar for any future models.

Here we use the familiar Scikit-Learn modeling syntax: we first create the model, then we train the model using `.fit` and then we make predictions on the testing data using `.predict_proba` (remember that we want probabilities and not a 0 or 1).

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

Now that the model has been trained, we can use it to make predictions. We want to predict the probabilities of not paying a loan, so we use the model `predict.proba` method. This returns an m x 2 array where m is the number of observations. The first column is the probability of the target being 0 and the second column is the probability of the target being 1 (so for a single row, the two columns must sum to 1). We want the probability the loan is not repaid, so we will select the second column.

The following code makes the predictions and selects the correct column.

In [ ]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]

The predictions must be in the format shown in the `sample_submission.csv` file, where there are only two columns: `SK_ID_CURR` and `TARGET`. We will create a dataframe in this format from the test set and the predictions called `submit`.

In [ ]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

The predictions represent a probability between 0 and 1 that the loan will not be repaid. If we were using these predictions to classify applicants, we could set a probability threshold for determining that a loan is risky.

In [ ]:
# Save the submission to a csv file
submit.to_csv('log_reg_baseline.csv', index = False)

The submission has now been saved to the virtual environment in which our notebook is running. To access the submission, at the end of the notebook, we will hit the blue Commit & Run button at the upper right of the kernel. This runs the entire notebook and then lets us download any files that are created during the run.

Once we run the notebook, the files created are available in the Versions tab under the Output sub-tab. From here, the submission files can be submitted to the competition or downloaded. Since there are several models in this notebook, there will be multiple output files.

__The logistic regression baseline should score around 0.671 when submitted.__

## Improved Model: Random Forest

To try and beat the poor performance of our baseline, we can update the algorithm. Let's try using a Random Forest on the same training data to see how that affects performance. The Random Forest is a much more powerful model especially when we use hundreds of trees. We will use 100 trees in the random forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
# Train on the training data
random_forest.fit(train, train_labels)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline.csv', index = False)

These predictions will also be available when we run the entire notebook.

__This model should score around 0.678 when submitted.__

### Make Predictions using Engineered Features

The only way to see if the Polynomial Features and Domain knowledge improved the model is to train a test a model on these features! We can then compare the submission performance to that for the model without these features to gauge the effect of our feature engineering.

In [ ]:
poly_features_names = list(app_train_poly.columns)

# Impute the polynomial features
imputer = Imputer(strategy = 'median')

poly_features = imputer.fit_transform(app_train_poly)
poly_features_test = imputer.transform(app_test_poly)

# Scale the polynomial features
scaler = MinMaxScaler(feature_range = (0, 1))

poly_features = scaler.fit_transform(poly_features)
poly_features_test = scaler.transform(poly_features_test)

random_forest_poly = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
# Train on the training data
random_forest_poly.fit(poly_features, train_labels)

# Make predictions on the test data
predictions = random_forest_poly.predict_proba(poly_features_test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline_engineered.csv', index = False)

This model scored 0.678 when submitted to the competition, exactly the same as that without the engineered features. Given these results, it does not appear that our feature construction helped in this case.

#### Testing Domain Features

Now we can test the domain features we made by hand.

In [ ]:
app_train_domain = app_train_domain.drop(columns = 'TARGET')

domain_features_names = list(app_train_domain.columns)

# Impute the domainnomial features
imputer = Imputer(strategy = 'median')

domain_features = imputer.fit_transform(app_train_domain)
domain_features_test = imputer.transform(app_test_domain)

# Scale the domainnomial features
scaler = MinMaxScaler(feature_range = (0, 1))

domain_features = scaler.fit_transform(domain_features)
domain_features_test = scaler.transform(domain_features_test)

random_forest_domain = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

# Train on the training data
random_forest_domain.fit(domain_features, train_labels)

# Extract feature importances
feature_importance_values_domain = random_forest_domain.feature_importances_
feature_importances_domain = pd.DataFrame({'feature': domain_features_names, 'importance': feature_importance_values_domain})

# Make predictions on the test data
predictions = random_forest_domain.predict_proba(domain_features_test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline_domain.csv', index = False)

This scores 0.679 when submitted which probably shows that the engineered features do not help in this model (however they do help in the Gradient Boosting Model at the end of the notebook).

In later notebooks, we will do more [feature engineering](https://docs.featuretools.com/index.html) by using the information from the other data sources. From experience, this will definitely help our model!

## Model Interpretation: Feature Importances

As a simple method to see which variables are the most relevant, we can look at the feature importances of the random forest. Given the correlations we saw in the exploratory data analysis, we should expect that the most important features are the `EXT_SOURCE` and the `DAYS_BIRTH`. We may use these feature importances as a method of dimensionality reduction in future work.

In [ ]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better.

    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance

    Returns:
        shows a plot of the 15 most importance features

        df (dataframe): feature importances sorted by importance (highest to lowest)
        with a column for normalized importance
        """

    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()

    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()

    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))),
            df['importance_normalized'].head(15),
            align = 'center', edgecolor = 'k')

    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))

    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()

    return df

In [ ]:
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

As expected, the most important features are those dealing with `EXT_SOURCE` and `DAYS_BIRTH`. We see that there are only a handful of features with a significant importance to the model, which suggests we may be able to drop many of the features without a decrease in performance (and we may even see an increase in performance.) Feature importances are not the most sophisticated method to interpret a model or perform dimensionality reduction, but they let us start to understand what factors our model takes into account when it makes predictions.

In [ ]:
feature_importances_domain_sorted = plot_feature_importances(feature_importances_domain)

We see that all four of our hand-engineered features made it into the top 15 most important! This should give us confidence that our domain knowledge was at least partially on track.

# Conclusions

In this notebook, we saw how to get started with a Kaggle machine learning competition. We first made sure to understand the data, our task, and the metric by which our submissions will be judged. Then, we performed a fairly simple EDA to try and identify relationships, trends, or anomalies that may help our modeling. Along the way, we performed necessary preprocessing steps such as encoding categorical variables, imputing missing values, and scaling features to a range. Then, we constructed new features out of the existing data to see if doing so could help our model.

Once the data exploration, data preparation, and feature engineering was complete, we implemented a baseline model upon which we hope to improve. Then we built a second slightly more complicated model to beat our first score. We also carried out an experiment to determine the effect of adding the engineering variables.

We followed the general outline of a [machine learning project](https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-one-c62152f39420):

1.  Understand the problem and the data
2. Data cleaning and formatting (this was mostly done for us)
3. Exploratory Data Analysis
4. Baseline model
5.  Improved model
6. Model interpretation (just a little)

Machine learning competitions do differ slightly from typical data science problems in that we are concerned only with achieving the best performance on a single metric and do not care about the interpretation. However, by attempting to understand how our models make decisions, we can try to improve them or examine the mistakes in order to correct the errors. In future notebooks we will look at incorporating more sources of data, building more complex models (by following the code of others), and improving our scores.

I hope this notebook was able to get you up and running in this machine learning competition and that you are now ready to go out on your own - with help from the community - and start working on some great problems!

__Running the notebook__: now that we are at the end of the notebook, you can hit the blue Commit & Run button to execute all the code at once. After the run is complete (this should take about 10 minutes), you can then access the files that were created by going to the versions tab and then the output sub-tab. The submission files can be directly submitted to the competition from this tab or they can be downloaded to a local machine and saved. The final part is to share the share the notebook: go to the settings tab and change the visibility to Public. This allows the entire world to see your work!

### Follow-up Notebooks

For those looking to keep working on this problem, I have a series of follow-up notebooks:

* [Manual Feature Engineering Part One](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering)
* [Manual Feature Engineering Part Two](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering-p2)
* [Introduction to Automated Feature Engineering](https://www.kaggle.com/willkoehrsen/automated-feature-engineering-basics)
* [Advanced Automated Feature Engineering](https://www.kaggle.com/willkoehrsen/tuning-automated-feature-engineering-exploratory)
* [Feature Selection](https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection)
* [Intro to Model Tuning: Grid and Random Search](https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search)

As always, I welcome feedback and constructive criticism. I write for Towards Data Science at https://medium.com/@williamkoehrsen/ and can be reached on Twitter at https://twitter.com/koehrsen_will

Will


# Just for Fun: Light Gradient Boosting Machine

Now (if you want, this part is entirely optional) we can step off the deep end and use a real machine learning model: the [gradient boosting machine](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/) using the [LightGBM library](http://lightgbm.readthedocs.io/en/latest/Quick-Start.html)! The Gradient Boosting Machine is currently the leading model for learning on structured datasets (especially on Kaggle) and we will probably need some form of this model to do well in the competition. Don't worry, even if this code looks intimidating, it's just a series of small steps that build up to a complete model. I added this code just to show what may be in store for this project, and because it gets us a slightly better score on the leaderboard. In future notebooks we will see how to work with more advanced models (which mostly means adapting existing code to make it work better), feature engineering, and feature selection. See you in the next notebook!  

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):

    """Train and test a light gradient boosting model using
    cross validation.

    Parameters
    --------
        features (pd.DataFrame):
            dataframe of training features to use
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame):
            dataframe of testing features to use
            for making predictions with the model.
        encoding (str, default = 'ohe'):
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation

    Return
    --------
        submission (pd.DataFrame):
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame):
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame):
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.

    """

    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']

    # Extract the labels for training
    labels = features['TARGET']

    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])


    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)

        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)

        # No categorical indices to record
        cat_indices = 'auto'

    # Integer label encoding
    elif encoding == 'le':

        # Create a label encoder
        label_encoder = LabelEncoder()

        # List for storing categorical indices
        cat_indices = []

        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)

    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")

    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)

    # Extract feature names
    feature_names = list(features.columns)

    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)

    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))

    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])

    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])

    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []

    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):

        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]

        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary',
                                   class_weight = 'balanced', learning_rate = 0.05,
                                   reg_alpha = 0.1, reg_lambda = 0.1,
                                   subsample = 0.8, n_jobs = -1, random_state = 50)

        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)

        # Record the best iteration
        best_iteration = model.best_iteration_

        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits

        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits

        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]

        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']

        valid_scores.append(valid_score)
        train_scores.append(train_score)

        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()

    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})

    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})

    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)

    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))

    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')

    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores})

    return submission, feature_importances, metrics

In [ ]:
submission, fi, metrics = model(app_train, app_test)
print('Baseline metrics')
print(metrics)

In [ ]:
fi_sorted = plot_feature_importances(fi)

In [ ]:
submission.to_csv('baseline_lgb.csv', index = False)

This submission should score about 0.735 on the leaderboard. We will certainly best that in future work!

In [ ]:
app_train_domain['TARGET'] = train_labels

# Test the domain knolwedge features
submission_domain, fi_domain, metrics_domain = model(app_train_domain, app_test_domain)
print('Baseline with domain knowledge features metrics')
print(metrics_domain)

In [ ]:
fi_sorted = plot_feature_importances(fi_domain)

Again, we see tha some of our features made it into the most important. Going forward, we will need to think about whatother domain knowledge features may be useful for this problem (or we should consult someone who knows more about the financial industry!

In [ ]:
submission_domain.to_csv('baseline_lgb_domain_features.csv', index = False)

This model scores about 0.754 when submitted to the public leaderboard indicating that the domain features do improve the performance! [Feature engineering](https://en.wikipedia.org/wiki/Feature_engineering) is going to be a critical part of this competition (as it is for all machine learning problems)!